In [1]:
collab = 0

<!-- @format -->

# Ollama Step-Up

This section installs necessary packages, sets up the Ollama API, and runs the server in a separate thread.


In [2]:
from IPython.display import clear_output
import subprocess
import threading
import os

# Install necessary packages and start Ollama API server
!sudo apt-get install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh
!ollama pull llama3.1:8b
%pip install -U lightrag[ollama]

clear_output()

In [3]:
import os
import subprocess
import threading


def ollama():
    """
    Start the Ollama API server in a separate thread with appropriate environment settings based on the execution context.

    The function sets environment variables for the Ollama server and starts the server using the `subprocess.Popen` method.
    It configures the server to listen on different hosts based on whether the script is running in a Colab environment or not.

    Environment Variables:
    - 'OLLAMA_HOST': The address and port on which the Ollama server will listen.
    - 'OLLAMA_ORIGINS': The allowed origins for requests to the Ollama server.

    If running in a Colab environment, the server is set to listen on '0.0.0.0:11434'. Otherwise, it listens on '127.0.0.1:11434'.

    Note:
    - Ensure that the `ollama` command is available in the system path for `subprocess.Popen` to work correctly.
    - This function will not block the main thread as it runs the server in a separate thread.

    Returns:
    None
    """
    if collab:
        os.environ["OLLAMA_HOST"] = "0.0.0.0:11434"
        os.environ["OLLAMA_ORIGINS"] = "*"
        subprocess.Popen(["ollama", "serve"])
    else:
        os.environ["OLLAMA_HOST"] = "127.0.0.1:11434"
        os.environ["OLLAMA_ORIGINS"] = "*"
        subprocess.Popen(["ollama", "serve"])


# Start Ollama API server in a separate thread
ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()
clear_output()

<!-- @format -->

### Setting Up Simple Question-Answering Component

This section sets up a simple QA component using the Ollama model.


In [4]:
# %%
from lightrag.core.generator import Generator
from lightrag.core.component import Component
from lightrag.core.model_client import ModelClient

# Template for the QA model
qa_template = r"""<SYS>
You are a helpful assistant.
</SYS>
User: {{input_str}}
You:"""

# Define the SimpleQA class


class SimpleQA(Component):
    """
    A simple Question-Answering (QA) component built on top of the lightrag framework.

    This class initializes a text generation component using the provided model client,
    model configurations, and a predefined QA template. The component is designed to
    generate responses to user input by following the QA template.

    Attributes:
    - generator (Generator): An instance of the Generator class, responsible for
      generating text based on the model, input data, and template.

    Methods:
    - call(input: dict) -> str: Synchronously generate a response based on the user input.
    - acall(input: dict) -> str: Asynchronously generate a response based on the user input.
    """

    def __init__(self, model_client: ModelClient, model_kwargs: dict):
        super().__init__()
        self.generator = Generator(
            model_client=model_client,
            model_kwargs=model_kwargs,
            template=qa_template,
        )

    def call(self, input: dict) -> str:
        """
        Synchronously generate a response based on the user input.

        Parameters:
        - input (dict): A dictionary containing the input string under the key 'input_str'.

        Returns:
        - str: The generated response as a string.
        """
        return self.generator.call({"input_str": str(input)})

    async def acall(self, input: dict) -> str:
        """
        Asynchronously generate a response based on the user input.

        Parameters:
        - input (dict): A dictionary containing the input string under the key 'input_str'.

        Returns:
        - str: The generated response as a string.
        """
        return await self.generator.acall({"input_str": str(input)})

<!-- @format -->

### Running a Simple QA Example

This section demonstrates how to use the SimpleQA component to answer a basic question.


In [5]:
from lightrag.components.model_client import OllamaClient
from IPython.display import Markdown, display

# Initialize the SimpleQA component with the Ollama model
model = {
    "model_client": OllamaClient(),  # Instantiate the model client for Ollama
    # Specify the model and its version to be used
    "model_kwargs": {"model": "llama3.1:8b"},
}
# Create an instance of SimpleQA with the provided model configuration
simple_qa = SimpleQA(**model)

# Ask a simple question and display the answer
# Generate a response using the SimpleQA component
output = simple_qa("what is 2+2")
# Display the answer in Markdown format
display(Markdown(f"**Answer:** {output.data}"))

**Answer:** That's an easy one!

The answer to 2 + 2 is... 4!

<!-- @format -->

# Constants


<!-- @format -->

### Ignore List

This section defines the list of directories, files, and file extensions to be ignored when processing the repository.


In [6]:
# List of file extensions to be ignored when constructuing folder structure,
# as they are unlikely to contain relevant information.
ignore_list_folder_structure = [
    # General
    ".git",  # Git repository metadata
    "node_modules",  # Node.js modules
    ".idea",  # JetBrains IDE project files
    ".vscode",  # Visual Studio Code settings
    "__pycache__",  # Python bytecode cache
    ".DS_Store",  # macOS directory metadata
    ".env",  # Environment variable files
    "venv",  # Python virtual environment
    "build",  # Build output directories
    "dist",  # Distribution directories
    "target",  # Output from Java and Rust builds
    ".pytest_cache",  # Pytest cache files
    "*.log",  # Log files
    "*.tmp",  # Temporary files
    # Python
    "*.pyc",  # Compiled Python files
    ".mypy_cache",  # Mypy type checker cache
    ".tox",  # Tox environment
    # JavaScript/Node.js
    "npm-debug.log",  # NPM debug logs
    "yarn-error.log",  # Yarn error logs
    ".parcel-cache",  # Parcel bundler cache
    "coverage",  # Code coverage reports
    ".next",  # Next.js build directory
    "out",  # Output directory for Next.js
    # Java
    "*.class",  # Compiled Java classes
    "*.jar",  # JAR files
    "*.war",  # WAR files
    ".settings",  # Eclipse settings
    ".classpath",  # Eclipse classpath
    ".project",  # Eclipse project file
    # C/C++
    "*.o",  # Object files
    "*.a",  # Static libraries
    "*.so",  # Shared libraries
    "*.out",  # Executable files
    "*.exe",  # Windows executables
    "CMakeFiles",  # CMake build files
    "CMakeCache.txt",  # CMake cache
    "*.dSYM",  # macOS debug symbols
    "*.pdb",  # Windows debug symbols
    # Rust
    "*.rlib",  # Rust libraries
    "Cargo.lock",  # Cargo lock file
    # Go
    "bin",  # Binary output directory
    "pkg",  # Package output directory
    "*.test",  # Go test binaries
    "vendor",  # Vendor directory (if not used)
    # Ruby
    ".bundle",  # Bundler directory
    "vendor/bundle",  # Bundled gems
    "log",  # Log files
    "tmp",  # Temporary files
    ".gem",  # RubyGems metadata
    # PHP
    "vendor",  # Composer dependencies
    ".phpunit.result.cache",  # PHPUnit result cache
    # Android
    ".gradle",  # Gradle files
    "*.apk",  # Android package
    "*.ap_ ",  # Android resources package
    "local.properties",  # Android SDK settings
    # .NET/C#
    "bin",  # Binary output directory
    "obj",  # Object files directory
    "*.dll",  # DLL files
    "*.user",  # User settings
    "packages",  # NuGet packages
    # LaTeX
    "*.aux",  # Auxiliary files
    "*.toc",  # Table of contents
    "*.out",  # Auxiliary output files
    "*.synctex.gz",  # SyncTeX file
    "*.fls",  # LaTeX build files
    "*.fdb_latexmk",  # LaTeX build files
]


# List of file extensions to be ignored based on file types:
ignore_list_extensions = [
    # Image formats
    ".png",
    ".jpg",
    ".jpeg",
    ".gif",
    ".bmp",
    ".svg",
    ".tiff",
    ".webp",
    ".heif",
    ".heic",
    ".ico",
    ".raw",
    ".psd",
    # Audio formats
    ".mp3",
    ".wav",
    ".flac",
    ".aac",
    ".ogg",
    ".m4a",
    ".wma",
    ".aiff",
    ".alac",
    ".pcm",
    # Video formats
    ".mp4",
    ".avi",
    ".mkv",
    ".mov",
    ".wmv",
    ".flv",
    ".webm",
    ".m4v",
    ".mpg",
    ".mpeg",
    ".3gp",
    ".ogv",
    ".rm",
    ".swf"
    # Binary files
    ".exe",
    ".dll",
    ".bin",
    ".iso",
    ".img",
    # System files
    ".sys",
    ".log",
    ".bak",
    ".tmp",
    ".ini",
    # Font files
    ".ttf",
    ".otf",
    ".woff",
    ".woff2",
    # Miscellaneous
    ".ico",
    ".svg",
    ".pdf",
    ".doc",
    ".docx",
    ".xls",
    ".xlsx",
    ".ppt",
    ".pptx",
]

# List of additional file extensions to be ignored when searching for API references,
# as they are unlikely to contain relevant API information.

api_additional_extensions = [
    # Text and document formats
    ".copy",
    ".local",
    ".json",
    ".config",
    ".md",
    ".txt",
    ".log",
    ".yml",
    ".yaml",
    ".xml",
    ".ini",
    ".pdf",
    ".csv",
    ".tsv",
    # Font formats
    ".woff",
    ".woff2",
    ".ttf",
    ".eot",
    ".otf",
    # Configuration and map files
    ".config.ts",
    ".map",
    ".lock",
    # Styling files
    ".css",
    ".scss",
    ".sass",
    ".less",
    ".styl",
    ".pcss",
    ".postcss",
]

# Combine ignore_extensions and api_additional_extensions

api_ignore_extensions = ignore_list_extensions + api_additional_extensions

# Additional specific ignores for API references
specific_ignores_api = [".gitignore", ".config.js", ".config.ts"]

<!-- @format -->

### Project Icons

This section defines the map of the project type and project icons


In [7]:
project_icons = {
    "ecommerce": {
        "icon": "https://img.icons8.com/nolan/512/1A6DFF/C822FF/shopping-basket-2.png"
    },
    "banking": {"icon": "https://img.icons8.com/nolan/512/1A6DFF/C822FF/bank.png"},
    "school": {"icon": "https://img.icons8.com/nolan/512/1A6DFF/C822FF/school.png"},
    "education": {
        "icon": "https://img.icons8.com/nolan/512/1A6DFF/C822FF/graduation-cap.png"
    },
    "work": {"icon": "https://img.icons8.com/nolan/512/1A6DFF/C822FF/briefcase.png"},
    "healthcare": {
        "icon": "https://img.icons8.com/nolan/512/1A6DFF/C822FF/hospital-room.png"
    },
    "real_estate": {"icon": "https://img.icons8.com/nolan/512/1A6DFF/C822FF/home.png"},
    "travel": {"icon": "https://img.icons8.com/nolan/512/1A6DFF/C822FF/passport.png"},
    "social_media": {
        "icon": "https://img.icons8.com/nolan/512/1A6DFF/C822FF/share.png"
    },
    "fitness": {"icon": "https://img.icons8.com/nolan/512/1A6DFF/C822FF/dumbbell.png"},
    "news": {"icon": "https://img.icons8.com/nolan/512/1A6DFF/C822FF/news.png"},
    "entertainment": {
        "icon": "https://img.icons8.com/nolan/512/1A6DFF/C822FF/clapperboard.png"
    },
    "food_delivery": {
        "icon": "https://img.icons8.com/nolan/512/1A6DFF/C822FF/food-delivery.png"
    },
    "finance": {"icon": "https://img.icons8.com/nolan/512/1A6DFF/C822FF/money.png"},
    "transportation": {
        "icon": "https://img.icons8.com/nolan/512/1A6DFF/C822FF/bus.png"
    },
    "hospitality": {"icon": "https://img.icons8.com/nolan/512/1A6DFF/C822FF/hotel.png"},
    "music": {
        "icon": "https://img.icons8.com/nolan/512/1A6DFF/C822FF/musical-notes.png"
    },
    "gaming": {"icon": "https://img.icons8.com/nolan/512/1A6DFF/C822FF/controller.png"},
    "environment": {
        "icon": "https://img.icons8.com/nolan/512/1A6DFF/C822FF/earth-planet.png"
    },
    "nonprofit": {"icon": "https://img.icons8.com/nolan/512/1A6DFF/C822FF/charity.png"},
    "photography": {
        "icon": "https://img.icons8.com/nolan/512/1A6DFF/C822FF/camera.png"
    },
}

<!-- @format -->

### Language, Framework, Tools Extension List

This section defines the list of Language, Framework, Tools Extension List


In [8]:
# Define known file extensions and configurations for different languages and tools
extensions = {
    "Python": [".py"],
    "JavaScript": [".js", ".jsx", ".ts", ".tsx"],
    "Java": [".java", ".class", ".jar", ".xml"],
    "C++": [".c", ".cpp", ".h", ".hpp", ".cc", ".cxx"],
    "C#": [".cs", ".csproj"],
    "Ruby": [".rb", ".gemspec", ".ru"],
    "PHP": [".php", ".phtml"],
    "Swift": [".swift"],
    "Go": [".go"],
    "Rust": [".rs"],
    "Kotlin": [".kt", ".kts"],
    "R": [".R", ".r", ".rmd"],
    "SQL": [".sql"],
    "HTML5": [".html", ".htm", ".css"],
    "CSS3": [".css"],
    "TypeScript": [".ts", ".tsx"],
    "Scala": [".scala", ".sbt"],
    "Perl": [".pl", ".pm"],
    "Objective-C": [".m", ".h"],
    "Shell": [".sh", ".bash", ".zsh"],
    "PowerShell": [".ps1"],
    "Haskell": [".hs", ".cabal"],
    "Lua": [".lua"],
    "Erlang": [".erl", ".hrl"],
    "Groovy": [".groovy", ".gvy"],
    "VHDL": [".vhdl", ".vhd"],
}
frameworks_extensions = {
    "React": [".jsx", ".tsx"],
    "Angular": ["angular.json"],
    "Vue.js": [".vue"],
    "Django": ["settings.py", "urls.py"],
    "Flask": [".py"],
    "Spring": [".xml"],
    "Maven": ["pom.xml"],
    "Gradle": ["build.gradle", "settings.gradle"],
    "Rails": ["Gemfile", "config.ru"],
    "Laravel": ["composer.json"],
    "Symfony": ["composer.json"],
    "Next.js": ["next.config.js"],
    "Gatsby": ["gatsby-config.js", "gatsby-node.js"],
    "Svelte": [".svelte"],
    "Bootstrap": ["bootstrap.min.css", "bootstrap.min.js"],
    "Jasmine": ["jasmine.json"],
    "Mocha": ["mocha.opts"],
    "Express": ["app.js", "server.js"],
    "Sails.js": ["config/", "api/"],
    "ASP.NET": [".cshtml", ".vbhtml", "web.config"],
    "Spring Boot": ["application.properties", "application.yml"],
    "Quasar": ["quasar.conf.js"],
    "Electron": ["main.js", "package.json"],
}
tools_extensions = {
    "Webpack": ["webpack.config.js"],
    "Docker": ["Dockerfile"],
    "CI/CD": [".github/workflows/ci.yml", ".gitlab-ci.yml", "Jenkinsfile"],
    "Babel": [".babelrc", "babel.config.json"],
    "ESLint": [".eslintrc", ".eslintrc.js", ".eslintrc.json"],
    "Prettier": [".prettierrc", ".prettierrc.json"],
    "Jest": ["jest.config.js", "jest.config.json"],
    "Travis CI": [".travis.yml"],
    "CircleCI": [".circleci/config.yml"],
    "Appveyor": ["appveyor.yml"],
    "Composer": ["composer.json", "composer.lock"],
    "Puppet": ["manifest.pp", "Puppetfile"],
    "Ansible": ["ansible.cfg", "playbook.yml"],
    "Kubernetes": ["deployment.yaml", "service.yaml"],
    "Terraform": [".tf", "main.tf"],
}

<!-- @format -->

# Extraction of the needed data using Github API and Ollama


<!-- @format -->

### Package Installations

This section installs additional necessary Python packages.


In [9]:
from IPython.display import clear_output

# %pip install nest_asyncio
# %pip install prettytable
# %pip install tqdm
# %pip install -U lightrag[ollama]
# %pip install aiohttp
# %pip install pandas
# %pip install openpyxl
clear_output()

In [10]:
repository_url = "https://github.com/Eemayas/Daraz_Scraper"
# repository_url = "https://github.com/embraceitmobile/animated_tree_view"
# repository_url = "https://github.com/earthPerson-001/Simple-Pendulum-Simulation-Using-OpenGL"

<!-- @format -->

### Cloning GitHub Repo

This Section clone the repo into the targeted folder


In [11]:
from typing import Optional
import os
import subprocess
import shutil


async def clone_github_repo(
    repository_url: str, target_folder: str = "Github_repos"
) -> Optional[str]:
    """
    Clone a GitHub repository to a specific folder and remove the .git folder.

    Args:
        repository_url (str): The URL of the GitHub repository to clone.
        target_folder (str, optional): The target directory to clone the repository into. Defaults to 'Github_repos'.

    Returns:
        Optional[str]: The path to the cloned repository, or None if cloning failed.
    """
    # Extract the repository name from the URL
    repo_name = repository_url.split("/")[-1]
    target_path = os.path.join(target_folder, repo_name)

    if not os.path.exists(target_path):
        # Ensure the target folder exists
        os.makedirs(target_folder, exist_ok=True)
        print(f"Cloning repository from {repository_url} into {target_path}...")

        try:
            # Run the git clone command to clone the repository
            subprocess.run(["git", "clone", repository_url, target_path], check=True)
            print(f"Repository cloned into {target_path}/")

            # Remove the .git folder to clean up the cloned repository
            git_folder_path = os.path.join(target_path, ".git")
            if os.path.exists(git_folder_path):
                shutil.rmtree(git_folder_path)
                print(f"Removed .git folder from {target_path}/")

            # Return the path to the cloned repository
            return target_path
        except subprocess.CalledProcessError as e:
            # Handle errors that occur during the cloning process
            print(f"Error cloning repository: {e}")
            return None
    else:
        # If the repository folder already exists, skip the cloning process
        print(f"Repository folder '{target_path}' already exists. Skipping clone.")
        return target_path


# Clone the repository asynchronously
cloned_repo_path = await clone_github_repo(repository_url=repository_url)
print(cloned_repo_path)

Repository folder 'Github_repos\Daraz_Scraper' already exists. Skipping clone.
Github_repos\Daraz_Scraper


<!-- @format -->

### GitHub Metadata Extraction

This section defines functions to extract metadata from a GitHub repository.


In [12]:
import json
from prettytable import PrettyTable
import asyncio
import aiohttp
from typing import Any, Optional, List, Dict
from dataclasses import dataclass
import nest_asyncio

# Enable asyncio to be used in Jupyter Notebooks
nest_asyncio.apply()

# Define dataclasses for storing metadata


@dataclass
class Contributor:
    name: str
    profile_url: str
    avatar_url: str
    contributions: str


@dataclass
class RepositoryMetadata:
    # Define fields for repository metadata
    id: int
    node_id: str
    name: str
    full_name: str
    private: bool
    owner: Dict[str, Any]
    html_url: str
    description: Optional[str]
    fork: bool
    url: str
    forks_url: str
    keys_url: str
    collaborators_url: str
    teams_url: str
    hooks_url: str
    issue_events_url: str
    events_url: str
    assignees_url: str
    branches_url: str
    tags_url: str
    blobs_url: str
    git_tags_url: str
    git_refs_url: str
    trees_url: str
    statuses_url: str
    languages_url: str
    stargazers_url: str
    contributors_url: str
    subscribers_url: str
    subscription_url: str
    commits_url: str
    git_commits_url: str
    comments_url: str
    issue_comment_url: str
    contents_url: str
    compare_url: str
    merges_url: str
    archive_url: str
    downloads_url: str
    issues_url: str
    pulls_url: str
    milestones_url: str
    notifications_url: str
    labels_url: str
    releases_url: str
    deployments_url: str
    created_at: str
    updated_at: str
    pushed_at: str
    git_url: str
    ssh_url: str
    clone_url: str
    svn_url: str
    homepage: Optional[str]
    size: int
    stargazers_count: int
    watchers_count: int
    language: Optional[str]
    has_issues: bool
    has_projects: bool
    has_downloads: bool
    has_wiki: bool
    has_pages: bool
    has_discussions: bool
    forks_count: int
    mirror_url: Optional[str]
    archived: bool
    disabled: bool
    open_issues_count: int
    license: Optional[Dict[str, Any]]
    allow_forking: bool
    is_template: bool
    web_commit_signoff_required: bool
    topics: List[str]
    visibility: str
    forks: int
    open_issues: int
    watchers: int
    default_branch: str
    temp_clone_token: Optional[str]
    network_count: int
    subscribers_count: int
    contributors: List[Contributor]


# Parse repository metadata and contributors


def _parse_repository_metadata(
    repo_data: dict, contributors: List[Contributor]
) -> RepositoryMetadata:
    """
    Parse the JSON data returned by the GitHub API into a RepositoryMetadata object.

    Args:
        repo_data (dict): The repository metadata returned by the GitHub API.
        contributors (List[Contributor]): A list of Contributor objects.

    Returns:
        RepositoryMetadata: An object containing the parsed repository metadata.
    """
    owner_info = repo_data.get("owner", {}) or {}
    license_info = repo_data.get("license", {}) or {}

    return RepositoryMetadata(
        id=repo_data.get("id", 0),
        node_id=repo_data.get("node_id", ""),
        name=repo_data.get("name", ""),
        full_name=repo_data.get("full_name", ""),
        private=repo_data.get("private", False),
        owner=owner_info,
        html_url=repo_data.get("html_url", ""),
        description=repo_data.get("description", ""),
        fork=repo_data.get("fork", False),
        url=repo_data.get("url", ""),
        forks_url=repo_data.get("forks_url", ""),
        keys_url=repo_data.get("keys_url", ""),
        collaborators_url=repo_data.get("collaborators_url", ""),
        teams_url=repo_data.get("teams_url", ""),
        hooks_url=repo_data.get("hooks_url", ""),
        issue_events_url=repo_data.get("issue_events_url", ""),
        events_url=repo_data.get("events_url", ""),
        assignees_url=repo_data.get("assignees_url", ""),
        branches_url=repo_data.get("branches_url", ""),
        tags_url=repo_data.get("tags_url", ""),
        blobs_url=repo_data.get("blobs_url", ""),
        git_tags_url=repo_data.get("git_tags_url", ""),
        git_refs_url=repo_data.get("git_refs_url", ""),
        trees_url=repo_data.get("trees_url", ""),
        statuses_url=repo_data.get("statuses_url", ""),
        languages_url=repo_data.get("languages_url", ""),
        stargazers_url=repo_data.get("stargazers_url", ""),
        contributors_url=repo_data.get("contributors_url", ""),
        subscribers_url=repo_data.get("subscribers_url", ""),
        subscription_url=repo_data.get("subscription_url", ""),
        commits_url=repo_data.get("commits_url", ""),
        git_commits_url=repo_data.get("git_commits_url", ""),
        comments_url=repo_data.get("comments_url", ""),
        issue_comment_url=repo_data.get("issue_comment_url", ""),
        contents_url=repo_data.get("contents_url", ""),
        compare_url=repo_data.get("compare_url", ""),
        merges_url=repo_data.get("merges_url", ""),
        archive_url=repo_data.get("archive_url", ""),
        downloads_url=repo_data.get("downloads_url", ""),
        issues_url=repo_data.get("issues_url", ""),
        pulls_url=repo_data.get("pulls_url", ""),
        milestones_url=repo_data.get("milestones_url", ""),
        notifications_url=repo_data.get("notifications_url", ""),
        labels_url=repo_data.get("labels_url", ""),
        releases_url=repo_data.get("releases_url", ""),
        deployments_url=repo_data.get("deployments_url", ""),
        created_at=repo_data.get("created_at", ""),
        updated_at=repo_data.get("updated_at", ""),
        pushed_at=repo_data.get("pushed_at", ""),
        git_url=repo_data.get("git_url", ""),
        ssh_url=repo_data.get("ssh_url", ""),
        clone_url=repo_data.get("clone_url", ""),
        svn_url=repo_data.get("svn_url", ""),
        homepage=repo_data.get("homepage", ""),
        size=repo_data.get("size", 0),
        stargazers_count=repo_data.get("stargazers_count", 0),
        watchers_count=repo_data.get("watchers_count", 0),
        language=repo_data.get("language", ""),
        has_issues=repo_data.get("has_issues", False),
        has_projects=repo_data.get("has_projects", False),
        has_downloads=repo_data.get("has_downloads", False),
        has_wiki=repo_data.get("has_wiki", False),
        has_pages=repo_data.get("has_pages", False),
        has_discussions=repo_data.get("has_discussions", False),
        forks_count=repo_data.get("forks_count", 0),
        mirror_url=repo_data.get("mirror_url", None),
        archived=repo_data.get("archived", False),
        disabled=repo_data.get("disabled", False),
        open_issues_count=repo_data.get("open_issues_count", 0),
        license=license_info,
        allow_forking=repo_data.get("allow_forking", False),
        is_template=repo_data.get("is_template", False),
        web_commit_signoff_required=repo_data.get("web_commit_signoff_required", False),
        topics=repo_data.get("topics", []),
        visibility=repo_data.get("visibility", ""),
        forks=repo_data.get("forks", 0),
        open_issues=repo_data.get("open_issues", 0),
        watchers=repo_data.get("watchers", 0),
        default_branch=repo_data.get("default_branch", ""),
        temp_clone_token=repo_data.get("temp_clone_token", None),
        network_count=repo_data.get("network_count", 0),
        subscribers_count=repo_data.get("subscribers_count", 0),
        contributors=contributors,
    )


# Fetch repository metadata from GitHub API


async def _fetch_repository_metadata(
    session: aiohttp.ClientSession, url: str
) -> dict[str, Any]:
    """
    Fetch repository metadata from GitHub API.

    Args:
        session (aiohttp.ClientSession): The aiohttp session to use for the request.
        url (str): The API URL for the repository metadata.

    Returns:
        dict: The JSON response from the GitHub API containing the repository metadata.
    """
    async with session.get(url) as response:
        response.raise_for_status()
        return await response.json()


# Fetch contributors from GitHub API


async def _fetch_contributors(
    session: aiohttp.ClientSession, url: str
) -> List[Contributor]:
    """
    Fetch the list of contributors from the GitHub API.

    Args:
        session (aiohttp.ClientSession): The aiohttp session to use for the request.
        url (str): The API URL for the contributors.

    Returns:
        List[Contributor]: A list of Contributor objects containing contributor data.
    """
    async with session.get(url) as response:
        response.raise_for_status()
        contributors_data = await response.json()
        return [
            Contributor(
                name=contributor.get("login", ""),
                profile_url=contributor.get("html_url", ""),
                avatar_url=contributor.get("avatar_url", ""),
                contributions=str(contributor.get("contributions", "")),
            )
            for contributor in contributors_data
        ]


# Fetch and parse repository metadata including contributors


async def fetch_git_repository_metadata(
    session: aiohttp.ClientSession, repository_url: str
) -> Optional[RepositoryMetadata]:
    """
    Fetch and parse repository metadata from GitHub, including contributors.

    Args:
        session (aiohttp.ClientSession): The aiohttp session to use for the request.
        repository_url (str): The URL of the GitHub repository.

    Returns:
        Optional[RepositoryMetadata]: A RepositoryMetadata object with the repository data.
    """
    api_url = repository_url.replace(
        "https://github.com/", "https://api.github.com/repos/"
    )

    try:
        metadata = await _fetch_repository_metadata(session, api_url)
        contributors_url = metadata.get("contributors_url", "")
        contributors = (
            await _fetch_contributors(session, contributors_url)
            if contributors_url
            else []
        )
        return _parse_repository_metadata(metadata, contributors) if metadata else None
    except aiohttp.ClientError as exc:
        print(f"Client error while fetching repository metadata: {exc}")
        return None


# Print metadata in a formatted table


def print_metadata(metadata: RepositoryMetadata):
    """
    Print the repository metadata and contributors in a formatted table.

    Args:
        metadata (RepositoryMetadata): The metadata to print.
    """
    table_metadata = PrettyTable()
    table_metadata.field_names = ["Attribute", "Value"]

    for field in metadata.__dataclass_fields__:
        value = getattr(metadata, field)
        if isinstance(value, dict):
            value = json.dumps(value, indent=2)
        elif isinstance(value, list):
            value = ", ".join(str(item) for item in value)
        table_metadata.add_row([field, value])

    print(table_metadata)

    if metadata.contributors:
        contributors_table = PrettyTable()
        contributors_table.field_names = [
            "Contributor Name",
            "Profile URL",
            "Avatar URL",
            "No of Contributions",
        ]
        for contributor in metadata.contributors:
            contributors_table.add_row(
                [
                    contributor.name,
                    contributor.profile_url,
                    contributor.avatar_url,
                    contributor.contributions,
                ]
            )
        print(contributors_table)


# Main function to fetch and display repository metadata


async def main(repository_url: str):
    """
    Main function to fetch and display repository metadata.

    Args:
        repository_url (str): The URL of the GitHub repository to fetch metadata for.
    """
    async with aiohttp.ClientSession() as session:
        metadata = await fetch_git_repository_metadata(session, repository_url)
        if metadata:
            print_metadata(metadata)
        return metadata


# Fetch and display repository metadata
metadata = asyncio.run(main(repository_url))

+-----------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|          Attribute          |                                                                              Value                                                                              |
+-----------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|              id             |                                                                            707705001                                                                            |
|           node_id           |                                                                           R_kgDOKi64qQ                                                                          |
|             name            

<!-- @format -->

### Folder Structure

This section contains functions to print the folder structure of a cloned GitHub repository and to clone a repository.


In [13]:
from pathlib import Path
from typing import List, Optional


def print_folder_structure(
    dir_path: Path,
    level: int = -1,
    limit_to_directories: bool = False,
    length_limit: int = 1000,
    file_folder_to_be_ignored: List[str] = None,
) -> List[str]:
    """
    Generate a visual tree structure of the directory contents.

    Parameters:
    dir_path (Path): The root directory to start the tree from.
    level (int, optional): The depth of recursion. Defaults to -1 (no limit).
    limit_to_directories (bool, optional): If True, only directories are listed. Defaults to False.
    length_limit (int, optional): Limits the number of lines output. Defaults to 1000.
    file_folder_to_be_ignored (List[str], optional): A list of directory or file names to ignore. Defaults to None.

    Returns:
    List[str]: A list of strings representing the directory tree structure.
    """
    space = "    "
    branch = "│   "
    tee = "├── "
    last = "└── "
    dir_path = Path(dir_path)  # Ensure dir_path is a Path object
    files = 0
    directories = 0
    output = []

    if file_folder_to_be_ignored is None:
        file_folder_to_be_ignored = []

    def inner(dir_path: Path, prefix: str = "", level: int = -1):
        nonlocal files, directories
        if level == 0:
            return  # Stop recursion if level is 0
        if limit_to_directories:
            contents = [
                d
                for d in dir_path.iterdir()
                if d.is_dir() and d.name not in file_folder_to_be_ignored
            ]
        else:
            contents = [
                d for d in dir_path.iterdir() if d.name not in file_folder_to_be_ignored
            ]
        pointers = [tee] * (len(contents) - 1) + [last]
        for pointer, path in zip(pointers, contents):
            if path.is_dir():
                output.append(prefix + pointer + path.name + "/")
                directories += 1
                extension = branch if pointer == tee else space
                inner(path, prefix=prefix + extension, level=level - 1)
            elif not limit_to_directories:
                output.append(prefix + pointer + path.name)
                files += 1

    # Add the root directory name
    output.append(dir_path.name + "/")
    # Create an iterator from the inner function
    inner(dir_path, level=level)
    # Limit the output by length_limit
    if len(output) > length_limit:
        output = output[:length_limit]
        output.append(f"... length_limit, {length_limit}, reached, counted:")
    # Add the summary of directories and files
    output.append(
        f"\n{directories} directories" + (f", {files} files" if files else "")
    )

    return output


if cloned_repo_path:
    # Print the folder structure
    folder_structure = print_folder_structure(
        dir_path=Path(cloned_repo_path),
        file_folder_to_be_ignored=ignore_list_folder_structure,
    )
    folder_structure_str = "\n".join(folder_structure)
    folder_structure_markdown = (
        "# Folder Structure\n" + "```sh\n" + folder_structure_str + "\n" + "```"
    )
    print(folder_structure_markdown)
else:
    print("Repository cloning failed or was skipped.")

# Folder Structure
```sh
Daraz_Scraper/
├── .env.local.copy
├── .gitignore
├── app/
│   ├── api/
│   │   └── cron/
│   │       └── route.ts
│   ├── favicon.ico
│   ├── globals.css
│   ├── layout.tsx
│   ├── page.tsx
│   └── products/
│       └── [id]/
│           └── page.tsx
├── components/
│   ├── HeroCarousel.tsx
│   ├── Modal.tsx
│   ├── Navbar.tsx
│   ├── PriceInfoCard.tsx
│   ├── ProductCard.tsx
│   └── Searchbar.tsx
├── lib/
│   ├── action/
│   │   └── index.ts
│   ├── models/
│   │   └── product.model.ts
│   ├── mongoose.ts
│   ├── nodemailer/
│   │   └── index.ts
│   ├── scrapper/
│   │   └── index.ts
│   └── utils.ts
├── next.config.js
├── package-lock.json
├── package.json
├── postcss.config.js
├── public/
│   ├── assets/
│   │   ├── icons/
│   │   │   ├── arrow-down.svg
│   │   │   ├── arrow-right.svg
│   │   │   ├── arrow-up.svg
│   │   │   ├── bag.svg
│   │   │   ├── black-heart.svg
│   │   │   ├── bookmark.svg
│   │   │   ├── chart.svg
│   │   │   ├── check.svg
│   │   │

<!-- @format -->

### Model setup for Ollama


In [14]:
# Create the model selection
model = {"model_client": OllamaClient(), "model_kwargs": {"model": "llama3.1:8b"}}


def get_description_data(description):
    """
    Retrieve data from the description object if it has a `data` attribute.

    Parameters:
    description (object): The object from which to retrieve the data. This can be any object that may or may not have a `data` attribute.

    Returns:
    str: The data from the `description` object if it exists, otherwise the original description.
    """
    if hasattr(description, "data"):
        return description.data
    return description


def is_empty_or_error(description):
    """
    Check if the description is empty or contains an error message.

    Parameters:
    description (str): The description string to check for emptiness or error messages.

    Returns:
    bool: True if the description is empty or contains an HTTP 401 error message, otherwise False.
    """
    description_str = (
        get_description_data(description).strip()
        if isinstance(description, str)
        else ""
    )
    return not description_str or "HTTP error 401" in description_str

<!-- @format -->

### Summary Generation

This section contains code to generate summaries for files in a GitHub repository using a summarization model. The notebook includes functions to:

- Generate summaries for all relevant files in a repository.
- Handle errors and save results to an Excel file.
- Provide a mechanism to retry summarization for files that encountered errors.


In [15]:
from lightrag.core.generator import Generator
from lightrag.core.component import Component
from lightrag.components.model_client import OllamaClient

summary_template = r"""<SYS>
You are a summarization assistant specialized in coding files.
</SYS>
Please summarize the following code:
{{input_str}}
Summary:"""


class SummaryQA(Component):
    """
    A component that uses a summarization model to generate summaries for code.

    Parameters:
    model_client (OllamaClient): The model client to interact with the summarization model.
    model_kwargs (dict): Additional keyword arguments for the model.
    """

    def __init__(self, model_client: OllamaClient, model_kwargs: dict):  # type: ignore
        super().__init__()
        self.generator = Generator(
            model_client=model_client,
            model_kwargs=model_kwargs,
            template=summary_template,
        )

    def call(self, input: str) -> str:
        """
        Generate a summary for the provided code using the model.

        Parameters:
        input (str): The code to be summarized.

        Returns:
        str: The generated summary.
        """
        return self.generator.call({"input_str": input})

    async def acall(self, input: str) -> str:
        """
        Asynchronously generate a summary for the provided code using the model.

        Parameters:
        input (str): The code to be summarized.

        Returns:
        str: The generated summary.
        """
        return await self.generator.acall({"input_str": input})

In [16]:
import os
from pathlib import Path
from typing import List, Dict
from prettytable import PrettyTable
from tqdm import tqdm


def generate_summary(
    path: Path,
    ignore_list: List[str],
    summary_component: SummaryQA,
    ignore_extensions: List[str],
) -> List[Dict[str, str]]:
    """
    Generate a summary of files in the given path using the summarization model.

    Parameters:
    path (Path): The root directory to start scanning for files.
    ignore_list (List[str]): A list of directory or file names to ignore.
    summary_component (SummaryQA): The summarization component to use for generating summaries.
    ignore_extensions (List[str]): A list of file extensions to ignore.

    Returns:
    List[Dict[str, str]]: A list of dictionaries where each dictionary contains the file path and its summary.
    """
    summary = []
    files_to_process = []

    for root, dirs, files in os.walk(path):
        # Get relative path for the current directory
        relative_root = os.path.relpath(root, path)

        # Check if the directory should be ignored
        if any(ignored in relative_root.split(os.sep) for ignored in ignore_list):
            continue

        if relative_root == ".":
            summary.append({"file": "Modules", "description": "."})
        else:
            summary.append({"file": relative_root, "description": "Not a File"})

        # List files in the current directory
        for file in files:
            file_path = Path(root) / file

            # Check if the file should be ignored
            if any(ignored in file_path.parts for ignored in ignore_list):
                continue

            # Check if the file has an extension that should be skipped
            if file_path.suffix.lower() in ignore_extensions:
                continue

            files_to_process.append(file_path)

    # Use tqdm to display progress
    pbar = tqdm(files_to_process, unit="file")
    for file_path in pbar:
        # Update the description dynamically
        pbar.set_description(f"Processing files - {file_path}")
        try:
            # Read file content
            with open(file_path, "r") as f:
                file_content = f.read()

            # Generate summary using the model
            summary_text = summary_component.call(file_content)
            summary.append({"file": file_path, "description": summary_text})
        except Exception as e:
            summary.append(
                {"file": file_path, "description": f"Error processing file: {str(e)}"}
            )

    return summary


summary_qa = SummaryQA(**model)

if cloned_repo_path:
    path = Path(cloned_repo_path)
    if not path.is_dir():
        print(f"The path {path} is not a directory.")
    summary = generate_summary(
        path,
        ignore_list=ignore_list_folder_structure,
        summary_component=summary_qa,
        ignore_extensions=ignore_list_extensions,
    )
    table_summary = PrettyTable()
    table_summary.field_names = ["File", "Description"]

    for item in summary:
        table_summary.add_row([item["file"], item["description"]])

    print(table_summary)
else:
    print("Repository cloning failed or was skipped.")

Processing files - Github_repos\Daraz_Scraper\types\index.ts: 100%|██████████| 28/28 [02:18<00:00,  4.93s/file]              

+---------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

<!-- @format -->

#### Save Summary to Excel

This section converts the summary data into an Excel file format for further analysis or record-keeping.


In [47]:
import pandas as pd
from prettytable import PrettyTable


def save_summary_to_excel_and_print_table(
    summary: List[Dict[str, str]], cloned_repo_path: str
) -> None:
    """
    Save the summary data to an Excel file and print it in a formatted table.

    Parameters:
    summary (List[Dict[str, str]]): A list of dictionaries containing file paths and their summaries.
    cloned_repo_path (str): The path of the cloned repository to be used for naming the Excel file.

    Returns:
    None
    """
    if summary:
        # Initialize PrettyTable
        table_summary = PrettyTable()
        table_summary.field_names = ["File", "Description"]

        # Create a list to hold data for the DataFrame
        summary_data_for_excel = []

        for item in summary:
            # Retrieve description data
            description_data = get_description_data(item["description"])
            # Add data to PrettyTable
            table_summary.add_row([item["file"], description_data])
            # Add data to the list for the DataFrame
            summary_data_for_excel.append(
                {"File": item["file"], "Description": description_data}
            )

        # Print the PrettyTable
        print(table_summary)

        # Convert the list to a DataFrame
        summary_df = pd.DataFrame(summary_data_for_excel)

        # Define the path and name for the Excel file
        excel_path = f"output/{metadata.name}_summary.xlsx"

        # Save DataFrame to an Excel file
        summary_df.to_excel(excel_path, index=False, engine="openpyxl")
        print(f"Summary saved to {excel_path}")
    else:
        print("No summary data available to save or print.")


# Assuming `summary` and `cloned_repo_path` are defined earlier in the code
save_summary_to_excel_and_print_table(summary, cloned_repo_path)

+---------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                           File                          |                                                                                                                                                                                    Description                                                                                                                                                                                    |
+---------------------------------------------------------+-----------------------------------------------------------------------------

<!-- @format -->

#### Retry Summarization for Errors

This section handles cases where summaries couldn't be generated initially, either due to errors or blank descriptions.


In [18]:
from prettytable import PrettyTable

# Check if there is any summary data available
if summary:

    # Variable to store rows with empty or error descriptions before adding to the table
    blank_error_summary = []

    # Iterate over each item in the summary
    for item in summary:

        # Retrieve the description data from the item
        description_data = get_description_data(item["description"])

        # Check if the description is empty or contains an error
        if is_empty_or_error(description_data):

            # Save the row data (file path and description) into a variable
            row = [item["file"], description_data]

            # Append the row to the list of blank or error summaries
            blank_error_summary.append(row)

    # Initialize PrettyTable to format the output
    retry_table = PrettyTable()

    # Set the field names (column headers) for the table
    retry_table.field_names = ["File", "Description"]

    # Add the saved rows (those with blank or error descriptions) to the table
    for row in blank_error_summary:

        retry_table.add_row(row)

    # Print the PrettyTable with the rows that have blank or error descriptions
    print(retry_table)

else:
    # Print a message if no summary data is available
    print("NO SUMMARY DATA AVAILABLE")

+------+-------------+
| File | Description |
+------+-------------+
+------+-------------+


<!-- @format -->

#### Generate Summary for Specific Files

This section provides functionality to manually generate or update summaries for individual files based on user input.


In [19]:
from typing import List, Dict
from pathlib import Path
from prettytable import PrettyTable


def generate_summary_for_file(
    file_path: Path, qa_component: SummaryQA, existing_summaries: List[Dict[str, str]]
) -> List[Dict[str, str]]:
    """
    Generate or update a summary for a single file using the model.

    Args:
        file_path (Path): The path to the file to be summarized.
        qa_component (SummaryQA): The component that generates the summary.
        existing_summaries (List[Dict[str, str]]): A list of existing summaries to update.

    Returns:
        List[Dict[str, str]]: The updated list of summaries including the new or updated summary for the file.
    """
    file_name = file_path.name
    updated = False

    # Check if the file summary already exists in the existing_summaries list
    for summary in existing_summaries:
        if summary["file"] == file_name:
            updated = True  # Mark as updated if the summary already exists
            break

    # If no summary exists for the file, add a new entry with an empty description
    if not updated:
        existing_summaries.append({"file": file_name, "description": ""})

    try:
        # Read the content of the file
        with open(file_path, "r") as f:
            file_content = f.read()

        # Generate a summary for the file content using the qa_component
        summary_text = qa_component.call(file_content)

        # Update the description in the existing_summaries list
        for summary in existing_summaries:
            if summary["file"] == file_name:
                summary["description"] = summary_text
                break
    except Exception as e:
        # Handle exceptions and update the summary with an error message
        for summary in existing_summaries:
            if summary["file"] == file_name:
                summary["description"] = f"Error processing file {file_path}: {str(e)}"
                break

    return existing_summaries


if blank_error_summary:
    # Initialize an empty list to store the summaries
    summaries = []

    # Prompt the user for a file path
    user_input = input("Please enter the path to the file you want to summarize: ")
    file_path = Path(user_input)

    if file_path.is_file():
        # Generate or update the summary for the specified file
        summaries = generate_summary_for_file(file_path, summary_qa, summaries)

        # Print the summary using PrettyTable
        table_summary = PrettyTable()
        table_summary.field_names = ["File", "Description"]

        # Add each summary to the PrettyTable
        for summary in summaries:
            table_summary.add_row([summary["file"], summary["description"]])

        print(table_summary)
    else:
        # If the path is not a valid file, print an error message
        print(f"The path {file_path} is not a valid file.")

<!-- @format -->

#### Combine Summaries

This section combines all generated summaries into a single summary string, filtering out unnecessary or erroneous data.


In [20]:
if summary:
    # Combine summaries, ignoring "Not a File" or error messages
    combined_summary = " ".join(
        [
            get_description_data(item["description"])
            for item in summary
            if get_description_data(item["description"])
            and get_description_data(item["description"]) != "Not a File"
            and get_description_data(item["description"]) != "."
            and not get_description_data(item["description"]).startswith(
                "HTTP error 401"
            )
        ]
    )
    print(combined_summary)

It seems you forgot to provide the code for me to summarize.

Please paste the code, and I'll be happy to assist you in summarizing it! This code is a `.gitignore` file, which tells Git to ignore certain files and directories in a project.

The summary is:

**Ignored Files/Directories:**

* Node.js dependencies (`node_modules`, `.pnp`, `.pnp.js`)
* Test coverage directory (`coverage`)
* Next.js build and output folders (`/.next/`, `/out/`, `/build`)
* Production build folder
* Mac-specific file (.DS_Store)
* SSL certificates (*.pem)
* Debug logs (npm, yarn, and error logs)
* Local environment files (.env*.local)
* Vercel settings file (.vercel)
* TypeScript build info file (`.tsbuildinfo`) Here is a summary of the code:

This is a `next.config.js` file for a Next.js project. It enables experimental features such as server-side rendering and external packages, specifically allowing the use of Mongoose. Additionally, it specifies image domains to be loaded from Daraz (static-01.daraz.com

<!-- @format -->

### Header

This section contains code to generate

- Project image and name.
- GitHub status badges.
- Language and framework badges.
- Combining all elements for the project header.


<!-- @format -->

#### Project Image


In [21]:
def get_project_icon():
    """
    Prompt the user to select a project type or enter a custom link/path for the project icon.

    Returns:
        str: The URL or path of the selected or custom project icon.
             Returns None if 'None' is selected or if the input is invalid.
    """
    # Display project type options to the user
    print("Select the project type:")
    for i, key in enumerate(project_icons.keys(), start=1):
        print(f"{i}. {key.capitalize()}")
    print(f"{len(project_icons) + 1}. Custom")
    print(f"{len(project_icons) + 2}. None")

    # Prompt the user to select an option
    choice = input("Enter the number corresponding to the project type: ")

    if choice.isdigit():  # Check if the input is a digit
        choice = int(choice)

        # Check if the choice corresponds to a predefined project type
        if 1 <= choice <= len(project_icons):
            selected_type = list(project_icons.keys())[choice - 1]
            icon_url = project_icons[selected_type]["icon"]  # Get the icon URL
            return icon_url

        # Check if the choice is 'Custom'
        elif choice == len(project_icons) + 1:
            custom_link = input("Enter the custom link/path: ")
            return custom_link

        # Check if the choice is 'None'
        elif choice == len(project_icons) + 2:
            return None

    # If input is invalid, display an error message and return None
    print("Invalid choice. Please try again.")
    return None


# Get the icon URL or custom path
icon_url = get_project_icon()

if icon_url:
    # Generate the Markdown for displaying the project icon
    project_image_markdown = f"""
<p align="center">
    <img src="{icon_url}" width="200" style="border-radius: 20px;" />
</p>
    """
else:
    # Set an empty Markdown string if no icon is selected
    project_image_markdown = ""

# Print the Markdown content (for debugging or display in a non-notebook environment)
print(project_image_markdown)

# Display the Markdown content (useful in Jupyter notebooks)
display(Markdown(project_image_markdown))

Select the project type:
1. Ecommerce
2. Banking
3. School
4. Education
5. Work
6. Healthcare
7. Real_estate
8. Travel
9. Social_media
10. Fitness
11. News
12. Entertainment
13. Food_delivery
14. Finance
15. Transportation
16. Hospitality
17. Music
18. Gaming
19. Environment
20. Nonprofit
21. Photography
22. Custom
23. None

<p align="center">
    <img src="https://img.icons8.com/nolan/512/1A6DFF/C822FF/shopping-basket-2.png" width="200" style="border-radius: 20px;" />
</p>
    



<p align="center">
    <img src="https://img.icons8.com/nolan/512/1A6DFF/C822FF/shopping-basket-2.png" width="200" style="border-radius: 20px;" />
</p>
    

<!-- @format -->

#### Project Name


In [22]:
project_name_markdown = f"""
<p align="center">
    <h1>{metadata.name}</h1>
</p>
"""
print(project_name_markdown)
display(Markdown(project_name_markdown))


<p align="center">
    <h1>Daraz_Scraper</h1>
</p>




<p align="center">
    <h1>Daraz_Scraper</h1>
</p>


<!-- @format -->

#### GitHub Status Badges


In [23]:
def update_github_badge_urls(github_repo_link, badges):
    """Update GitHub status badge URLs based on the selected badges.

    Args:
        github_repo_link (str): The GitHub repository link.
        badges (list): A list of badge identifiers to include.

    Returns:
        str: A string containing the HTML code for the selected badges.
    """
    # Extract the owner and repository name from the GitHub link
    repo_path = github_repo_link.rstrip("/").replace("https://github.com/", "")

    # Define the badge URLs
    badge_urls = {
        "license": f"https://img.shields.io/github/license/{repo_path}?style=flat&color=0080ff",
        "last-commit": f"https://img.shields.io/github/last-commit/{repo_path}?style=flat&logo=git&logoColor=white&color=0080ff",
        "repo-top-language": f"https://img.shields.io/github/languages/top/{repo_path}?style=flat&color=0080ff",
        "repo-language-count": f"https://img.shields.io/github/languages/count/{repo_path}?style=flat&color=0080ff",
        "build-status": f"https://img.shields.io/github/actions/workflow/status/{repo_path}/build.yml?branch=main&style=flat&color=0080ff",
        "open-issues": f"https://img.shields.io/github/issues/{repo_path}?style=flat&color=0080ff",
        "forks": f"https://img.shields.io/github/forks/{repo_path}?style=flat&color=0080ff",
        "stars": f"https://img.shields.io/github/stars/{repo_path}?style=flat&color=0080ff",
        "pull-requests": f"https://img.shields.io/github/issues-pr/{repo_path}?style=flat&color=0080ff",
        "contributors": f"https://img.shields.io/github/contributors/{repo_path}?style=flat&color=0080ff",
        "commit-activity": f"https://img.shields.io/github/commit-activity/m/{repo_path}?style=flat&color=0080ff",
        "code-size": f"https://img.shields.io/github/languages/code-size/{repo_path}?style=flat&color=0080ff",
        "repo-size": f"https://img.shields.io/github/repo-size/{repo_path}?style=flat&color=0080ff",
        "downloads": f"https://img.shields.io/github/downloads/{repo_path}/total?style=flat&color=0080ff",
        "sponsors": f"https://img.shields.io/github/sponsors/{repo_path}?style=flat&color=0080ff",
        "release-version": f"https://img.shields.io/github/v/release/{repo_path}?style=flat&color=0080ff",
        "coverage": f"https://img.shields.io/codecov/c/github/{repo_path}?style=flat&color=0080ff",
        "code-quality": f"https://img.shields.io/codeclimate/quality/a/{repo_path}?style=flat&color=0080ff",
        "dependencies": f"https://img.shields.io/david/{repo_path}?style=flat&color=0080ff",
        "dev-dependencies": f"https://img.shields.io/david/dev/{repo_path}?style=flat&color=0080ff",
        "security": f"https://img.shields.io/snyk/vulnerabilities/github/{repo_path}?style=flat&color=0080ff",
        "performance": f"https://img.shields.io/website?style=flat&color=0080ff&url=https%3A%2F%2Fexample.com",
        "activity": f"https://img.shields.io/github/commit-activity/y/{repo_path}?style=flat&color=0080ff",
        "documentation": f"https://img.shields.io/docsify/docs?style=flat&color=0080ff",
        "version": f"https://img.shields.io/github/v/tag/{repo_path}?style=flat&color=0080ff",
    }

    # Create the HTML string based on the selected badges
    badges_html = "\n".join(
        f'  <img src="{badge_urls[badge]}" alt="{badge}">'
        for badge in badges
        if badge in badge_urls
    )

    # Generate the final HTML template
    html_template = f"""
<p align="center">
{badges_html}
</p>
    """

    return html_template


# Define the selected badges
selected_badges = [
    "license",
    "last-commit",
    "repo-top-language",
    "repo-language-count",
    "build-status",
    "open-issues",
    "forks",
    "stars",
    "pull-requests",
    "contributors",
    "commit-activity",
    "code-size",
    "repo-size",
    "downloads",
    "sponsors",
    "release-version",
    "coverage",
    "code-quality",
    "dependencies",
    "dev-dependencies",
    "security",
    "performance",
    "activity",
    "documentation",
    "version",
]

# Update the GitHub badge URLs
github_badge_markdown = update_github_badge_urls(repository_url, selected_badges)

# Print and display the GitHub badge markdown
print(github_badge_markdown)
display(Markdown(github_badge_markdown))


<p align="center">
  <img src="https://img.shields.io/github/license/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="license">
  <img src="https://img.shields.io/github/last-commit/Eemayas/Daraz_Scraper?style=flat&logo=git&logoColor=white&color=0080ff" alt="last-commit">
  <img src="https://img.shields.io/github/languages/top/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="repo-top-language">
  <img src="https://img.shields.io/github/languages/count/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="repo-language-count">
  <img src="https://img.shields.io/github/actions/workflow/status/Eemayas/Daraz_Scraper/build.yml?branch=main&style=flat&color=0080ff" alt="build-status">
  <img src="https://img.shields.io/github/issues/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="open-issues">
  <img src="https://img.shields.io/github/forks/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="forks">
  <img src="https://img.shields.io/github/stars/Eemayas/Daraz_Scraper?style=flat&colo


<p align="center">
  <img src="https://img.shields.io/github/license/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="license">
  <img src="https://img.shields.io/github/last-commit/Eemayas/Daraz_Scraper?style=flat&logo=git&logoColor=white&color=0080ff" alt="last-commit">
  <img src="https://img.shields.io/github/languages/top/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="repo-top-language">
  <img src="https://img.shields.io/github/languages/count/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="repo-language-count">
  <img src="https://img.shields.io/github/actions/workflow/status/Eemayas/Daraz_Scraper/build.yml?branch=main&style=flat&color=0080ff" alt="build-status">
  <img src="https://img.shields.io/github/issues/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="open-issues">
  <img src="https://img.shields.io/github/forks/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="forks">
  <img src="https://img.shields.io/github/stars/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="stars">
  <img src="https://img.shields.io/github/issues-pr/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="pull-requests">
  <img src="https://img.shields.io/github/contributors/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="contributors">
  <img src="https://img.shields.io/github/commit-activity/m/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="commit-activity">
  <img src="https://img.shields.io/github/languages/code-size/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="code-size">
  <img src="https://img.shields.io/github/repo-size/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="repo-size">
  <img src="https://img.shields.io/github/downloads/Eemayas/Daraz_Scraper/total?style=flat&color=0080ff" alt="downloads">
  <img src="https://img.shields.io/github/sponsors/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="sponsors">
  <img src="https://img.shields.io/github/v/release/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="release-version">
  <img src="https://img.shields.io/codecov/c/github/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="coverage">
  <img src="https://img.shields.io/codeclimate/quality/a/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="code-quality">
  <img src="https://img.shields.io/david/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="dependencies">
  <img src="https://img.shields.io/david/dev/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="dev-dependencies">
  <img src="https://img.shields.io/snyk/vulnerabilities/github/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="security">
  <img src="https://img.shields.io/website?style=flat&color=0080ff&url=https%3A%2F%2Fexample.com" alt="performance">
  <img src="https://img.shields.io/github/commit-activity/y/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="activity">
  <img src="https://img.shields.io/docsify/docs?style=flat&color=0080ff" alt="documentation">
  <img src="https://img.shields.io/github/v/tag/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="version">
</p>
    

<!-- @format -->

#### Language and Framework Badges


In [24]:
import json
import requests
import os

languages_found = set()
frameworks_found = set()
tools_found = set()


def identify_project(root_folder):
    """
    Identify languages, frameworks, and tools used in the project based on file extensions and configuration files.

    Parameters:
    root_folder (str): The root directory of the project to scan.

    Updates:
    - languages_found: A set of languages identified from file extensions.
    - frameworks_found: A set of frameworks identified from configuration files.
    - tools_found: A set of tools identified from configuration files.
    """
    for root, _, files in os.walk(root_folder):
        for file in files:
            ext = os.path.splitext(file)[1]
            if ext in [e for exts in extensions.values() for e in exts]:
                for language, exts in extensions.items():
                    if ext in exts:
                        languages_found.add(language)
            for framework, config_files in frameworks_extensions.items():
                if file in config_files:
                    frameworks_found.add(framework)
            for tool, config_files in tools_extensions.items():
                if file in config_files:
                    tools_found.add(tool)

    print("Languages found:", ", ".join(languages_found) or "None")
    print("Frameworks found:", ", ".join(frameworks_found) or "None")
    print("Tools found:", ", ".join(tools_found) or "None")


def get_languages_from_github(languages_url):
    """
    Retrieve the list of languages used in the GitHub repository.

    Parameters:
    languages_url (str): The URL of the GitHub API endpoint to fetch language data.

    Returns:
    dict: A dictionary with languages as keys and their respective sizes as values.
    """
    response = requests.get(languages_url)
    response.raise_for_status()  # Raise an error for bad responses
    languages = response.json()
    return languages


def load_shields_data(filename):
    """
    Load shields.io icons data from a JSON file.

    Parameters:
    filename (str): The path to the JSON file containing shields.io icon data.

    Returns:
    dict: A dictionary with technologies as keys and their badge URLs and other metadata as values.
    """
    with open(filename, "r") as file:
        return json.load(file)


def get_shields_urls(technologies, shields_data):
    """
    Generate URLs for shields.io badges based on the technologies found.

    Parameters:
    technologies (set): A set of technology names for which badge URLs are to be generated.
    shields_data (dict): A dictionary containing badge URLs and metadata for various technologies.

    Returns:
    dict: A dictionary with technology names as keys and their badge URLs as values.
    """
    results = {}
    for tech in technologies:
        if tech in shields_data:
            url, _ = shields_data[tech]
            results[tech] = url.format("for-the-badge")
        else:
            fallback_url = f"https://img.shields.io/badge/{tech}-ED8B00?logo={tech}&logoColor=white"
            results[tech] = fallback_url
    return results


def generate_language_badges(urls_map):
    """
    Generate HTML for language badges.

    Parameters:
    urls_map (dict): A dictionary with technology names as keys and badge URLs as values.

    Returns:
    str: HTML string containing the badges.
    """
    badges_html = "\n".join(
        f'  <img src="{urls_map[language]}" alt="{language}">'
        for language in urls_map.keys()
    )

    html_template = f"""
<p align="center">
{badges_html}
</p>
    """

    return html_template


if os.path.isdir(cloned_repo_path):
    identify_project(cloned_repo_path)
    languages_from_github = get_languages_from_github(metadata.languages_url)
    for language in languages_from_github.keys():
        languages_found.add(language)
    # Load shields.io data
    shields_data = load_shields_data("./shieldsio_icons.json")

    # Combine results and search in shields.io data
    combined_results = {
        **{lang: None for lang in languages_found},
        **{framework: None for framework in frameworks_found},
        **{tool: None for tool in tools_found},
    }

    # Get URLs for combined results
    badges_urls_map = get_shields_urls(combined_results.keys(), shields_data)

    # Print results
    if badges_urls_map:
        print(badges_urls_map)
        language_badges_markdown = f"""
<p align="center">
    <em>Constructed using the following tools and technologies:</em>
</p>
{generate_language_badges(badges_urls_map)}
"""
        print(language_badges_markdown)
        display(Markdown(language_badges_markdown))
    else:
        print("No matching languages, frameworks, or tools found in shields.io data")
else:
    print("Invalid directory path")

Languages found: JavaScript, TypeScript, CSS3, HTML5
Frameworks found: Electron, Next.js
Tools found: None
{'CSS': 'https://img.shields.io/badge/CSS3-1572B6.svg?style=for-the-badge&logo=CSS3&logoColor=white', 'CSS3': 'https://img.shields.io/badge/CSS3-1572B6.svg?style=for-the-badge&logo=CSS3&logoColor=white', 'TypeScript': 'https://img.shields.io/badge/TypeScript-3178C6.svg?style=for-the-badge&logo=TypeScript&logoColor=white', 'HTML5': 'https://img.shields.io/badge/HTML5-E34F26.svg?style=for-the-badge&logo=HTML5&logoColor=white', 'JavaScript': 'https://img.shields.io/badge/JavaScript-F7DF1E.svg?style=for-the-badge&logo=JavaScript&logoColor=black', 'Electron': 'https://img.shields.io/badge/Electron-47848F.svg?style=for-the-badge&logo=Electron&logoColor=white', 'Next.js': 'https://img.shields.io/badge/Next.js-000000.svg?style=for-the-badge&logo=nextdotjs&logoColor=white'}

<p align="center">
    <em>Constructed using the following tools and technologies:</em>
</p>

<p align="center">
  <


<p align="center">
    <em>Constructed using the following tools and technologies:</em>
</p>

<p align="center">
  <img src="https://img.shields.io/badge/CSS3-1572B6.svg?style=for-the-badge&logo=CSS3&logoColor=white" alt="CSS">
  <img src="https://img.shields.io/badge/CSS3-1572B6.svg?style=for-the-badge&logo=CSS3&logoColor=white" alt="CSS3">
  <img src="https://img.shields.io/badge/TypeScript-3178C6.svg?style=for-the-badge&logo=TypeScript&logoColor=white" alt="TypeScript">
  <img src="https://img.shields.io/badge/HTML5-E34F26.svg?style=for-the-badge&logo=HTML5&logoColor=white" alt="HTML5">
  <img src="https://img.shields.io/badge/JavaScript-F7DF1E.svg?style=for-the-badge&logo=JavaScript&logoColor=black" alt="JavaScript">
  <img src="https://img.shields.io/badge/Electron-47848F.svg?style=for-the-badge&logo=Electron&logoColor=white" alt="Electron">
  <img src="https://img.shields.io/badge/Next.js-000000.svg?style=for-the-badge&logo=nextdotjs&logoColor=white" alt="Next.js">
</p>
    


<!-- @format -->

### Combining all for Header


In [25]:
# Combine various markdown sections to create the header

header_markdown = (
    project_image_markdown  # Markdown for the project image
    + "\n"
    + project_name_markdown  # Markdown for the project name
    + "\n"
    + github_badge_markdown  # Markdown for GitHub badges
    + "\n"
    + language_badges_markdown  # Markdown for language badges
    + "\n"
    + ""  # Add an empty string for consistency
)

# Print the combined markdown header
print(header_markdown)

# Display the combined markdown header in a Markdown format
display(Markdown(header_markdown))


<p align="center">
    <img src="https://img.icons8.com/nolan/512/1A6DFF/C822FF/shopping-basket-2.png" width="200" style="border-radius: 20px;" />
</p>
    

<p align="center">
    <h1>Daraz_Scraper</h1>
</p>


<p align="center">
  <img src="https://img.shields.io/github/license/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="license">
  <img src="https://img.shields.io/github/last-commit/Eemayas/Daraz_Scraper?style=flat&logo=git&logoColor=white&color=0080ff" alt="last-commit">
  <img src="https://img.shields.io/github/languages/top/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="repo-top-language">
  <img src="https://img.shields.io/github/languages/count/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="repo-language-count">
  <img src="https://img.shields.io/github/actions/workflow/status/Eemayas/Daraz_Scraper/build.yml?branch=main&style=flat&color=0080ff" alt="build-status">
  <img src="https://img.shields.io/github/issues/Eemayas/Daraz_Scraper?style=flat&color=0080ff" a


<p align="center">
    <img src="https://img.icons8.com/nolan/512/1A6DFF/C822FF/shopping-basket-2.png" width="200" style="border-radius: 20px;" />
</p>
    

<p align="center">
    <h1>Daraz_Scraper</h1>
</p>


<p align="center">
  <img src="https://img.shields.io/github/license/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="license">
  <img src="https://img.shields.io/github/last-commit/Eemayas/Daraz_Scraper?style=flat&logo=git&logoColor=white&color=0080ff" alt="last-commit">
  <img src="https://img.shields.io/github/languages/top/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="repo-top-language">
  <img src="https://img.shields.io/github/languages/count/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="repo-language-count">
  <img src="https://img.shields.io/github/actions/workflow/status/Eemayas/Daraz_Scraper/build.yml?branch=main&style=flat&color=0080ff" alt="build-status">
  <img src="https://img.shields.io/github/issues/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="open-issues">
  <img src="https://img.shields.io/github/forks/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="forks">
  <img src="https://img.shields.io/github/stars/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="stars">
  <img src="https://img.shields.io/github/issues-pr/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="pull-requests">
  <img src="https://img.shields.io/github/contributors/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="contributors">
  <img src="https://img.shields.io/github/commit-activity/m/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="commit-activity">
  <img src="https://img.shields.io/github/languages/code-size/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="code-size">
  <img src="https://img.shields.io/github/repo-size/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="repo-size">
  <img src="https://img.shields.io/github/downloads/Eemayas/Daraz_Scraper/total?style=flat&color=0080ff" alt="downloads">
  <img src="https://img.shields.io/github/sponsors/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="sponsors">
  <img src="https://img.shields.io/github/v/release/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="release-version">
  <img src="https://img.shields.io/codecov/c/github/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="coverage">
  <img src="https://img.shields.io/codeclimate/quality/a/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="code-quality">
  <img src="https://img.shields.io/david/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="dependencies">
  <img src="https://img.shields.io/david/dev/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="dev-dependencies">
  <img src="https://img.shields.io/snyk/vulnerabilities/github/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="security">
  <img src="https://img.shields.io/website?style=flat&color=0080ff&url=https%3A%2F%2Fexample.com" alt="performance">
  <img src="https://img.shields.io/github/commit-activity/y/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="activity">
  <img src="https://img.shields.io/docsify/docs?style=flat&color=0080ff" alt="documentation">
  <img src="https://img.shields.io/github/v/tag/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="version">
</p>
    

<p align="center">
    <em>Constructed using the following tools and technologies:</em>
</p>

<p align="center">
  <img src="https://img.shields.io/badge/CSS3-1572B6.svg?style=for-the-badge&logo=CSS3&logoColor=white" alt="CSS">
  <img src="https://img.shields.io/badge/CSS3-1572B6.svg?style=for-the-badge&logo=CSS3&logoColor=white" alt="CSS3">
  <img src="https://img.shields.io/badge/TypeScript-3178C6.svg?style=for-the-badge&logo=TypeScript&logoColor=white" alt="TypeScript">
  <img src="https://img.shields.io/badge/HTML5-E34F26.svg?style=for-the-badge&logo=HTML5&logoColor=white" alt="HTML5">
  <img src="https://img.shields.io/badge/JavaScript-F7DF1E.svg?style=for-the-badge&logo=JavaScript&logoColor=black" alt="JavaScript">
  <img src="https://img.shields.io/badge/Electron-47848F.svg?style=for-the-badge&logo=Electron&logoColor=white" alt="Electron">
  <img src="https://img.shields.io/badge/Next.js-000000.svg?style=for-the-badge&logo=nextdotjs&logoColor=white" alt="Next.js">
</p>
    



<!-- @format -->

### Project Overview


In [26]:
from lightrag.core.generator import Generator
from lightrag.core.component import Component
from lightrag.components.model_client import OllamaClient

# Define the template for generating a project overview
project_overview_template = r"""<SYS>
You are a summarization assistant specialized in project documentation.
</SYS>
Based on the provided file summaries:
{{input_str}},

Generate a concise and descriptive one-paragraph overview of the project, including:
1. What the project is about (2 sentences).
2. What the project does (more than 3 sentences).
3. The technologies used.
4. The key features of the project.

Don't add predescription and post description in the answer.
Summary:"""


class OverviewQA(Component):
    """
    Component for generating a project overview.

    This class uses a model to generate a summary of the project based on the provided file summaries.
    """

    def __init__(self, model_client: OllamaClient, model_kwargs: dict):  # type: ignore
        """
        Initialize the OverviewQA component.

        Parameters:
        model_client (OllamaClient): The client used to interact with the model.
        model_kwargs (dict): Additional keyword arguments for the model.
        """
        super().__init__()
        self.generator = Generator(
            model_client=model_client,
            model_kwargs=model_kwargs,
            template=project_overview_template,
        )

    def call(self, summaries: str) -> str:
        """
        Generate a project overview based on file summaries.

        Parameters:
        summaries (str): The file summaries to use for generating the overview.

        Returns:
        str: The generated project overview.
        """
        return self.generator.call({"input_str": summaries})

    async def acall(self, summaries: str) -> str:
        """
        Asynchronous version for generating a project overview.

        Parameters:
        summaries (str): The file summaries to use for generating the overview.

        Returns:
        str: The generated project overview.
        """
        return await self.generator.acall({"input_str": summaries})

In [27]:
def generate_project_overview(
    combined_summary: str, overview_component: OverviewQA
) -> str:
    """
    Generate a concise and descriptive overview of the project.

    This function uses the OverviewQA component to create a project overview based on the combined summaries of the project files.

    Parameters:
    combined_summary (str): A string containing the combined summaries of the project files.
    overview_component (OverviewQA): The component used to generate the project overview.

    Returns:
    str: The generated project overview.
    """
    return overview_component.generator.call({"input_str": combined_summary})


# Initialize OverviewQA component with model configuration
overview_qa = OverviewQA(**model)

if combined_summary:
    # Generate the project overview
    project_overview = generate_project_overview(
        combined_summary, overview_component=overview_qa
    )
    # Clean and format the project overview text
    project_overview = get_description_data(project_overview)
    project_overview_markdown = (
        "# Project Overview\n\n"
        + project_overview.strip().replace("\n\n", "\n\n").replace("  ", " ")
    )
    # Print and display the project overview
    print(project_overview_markdown)
    display(Markdown(project_overview_markdown))
else:
    print("No combined summary available to generate the project overview.")

# Project Overview

The project is an e-commerce platform that enables users to buy and sell products, with a focus on product pricing, reviews, and stock management. It allows sellers to list their products with detailed information, such as price history, discount rates, and seller ratings, while also enabling buyers to review and rate the products they purchase. The project provides features like automatic email notifications for new orders, low stock levels, and threshold met, allowing users to stay informed about important events related to their business or purchases.

The project uses Mongoose to interact with a MongoDB database, storing products in a collection called "products". It also utilizes Node.js, Axios, Cheerio, and Nodemailer libraries to handle product scraping from Daraz website, email generation, and sending emails based on notification types. The key features of the project include automated email notifications for various events, product pricing and review manage

# Project Overview

The project is an e-commerce platform that enables users to buy and sell products, with a focus on product pricing, reviews, and stock management. It allows sellers to list their products with detailed information, such as price history, discount rates, and seller ratings, while also enabling buyers to review and rate the products they purchase. The project provides features like automatic email notifications for new orders, low stock levels, and threshold met, allowing users to stay informed about important events related to their business or purchases.

The project uses Mongoose to interact with a MongoDB database, storing products in a collection called "products". It also utilizes Node.js, Axios, Cheerio, and Nodemailer libraries to handle product scraping from Daraz website, email generation, and sending emails based on notification types. The key features of the project include automated email notifications for various events, product pricing and review management, stock level tracking, and seller information display.

<!-- @format -->

### Key Features


In [28]:
from lightrag.core.generator import Generator
from lightrag.core.component import Component
from lightrag.components.model_client import OllamaClient

key_feature_template = r"""<SYS>
You are an expert computer engineer specializing in project documentation and coding, with advanced knowledge of various programming technologies.
</SYS>
Based on the provided file summaries:
{{input_str}},

Extract and list the key features (minimum 5 features) in a concise format. Each feature should include:
- Feature Name: A brief description of the feature and its significance.

Use the following format for listing the features:
- **Feature Name**: Description of the feature and its significance.

Ensure that the features are listed clearly and concisely, highlighting the most important aspects and functionalities that define the project’s value and just give me bulletin. No explanation need before and after bulletin likes "Here are the key features extracted from the provided code snippets:", "Let me know if you'd like me to help with anything else!"
Summary:
"""


class FeatureQA(Component):
    """Component for extracting key features of the project."""

    def __init__(self, model_client: OllamaClient, model_kwargs: dict):  # type: ignore
        """
        Initialize the FeatureQA component.

        Parameters:
        model_client (OllamaClient): The model client used for generating features.
        model_kwargs (dict): Additional model parameters.
        """
        super().__init__()
        self.generator = Generator(
            model_client=model_client,
            model_kwargs=model_kwargs,
            template=key_feature_template,
        )

    def call(self, summaries: str) -> str:
        """
        Extract key features based on file summaries.

        Parameters:
        summaries (str): A string containing the combined summaries of the project files.

        Returns:
        str: A list of key features extracted from the file summaries.
        """
        return self.generator.call({"input_str": summaries})

    async def acall(self, summaries: str) -> str:
        """
        Asynchronous version for extracting key features.

        Parameters:
        summaries (str): A string containing the combined summaries of the project files.

        Returns:
        str: A list of key features extracted from the file summaries.
        """
        return await self.generator.acall({"input_str": summaries})


def generate_key_feature(combined_summary: str, feature_component: FeatureQA) -> str:
    """
    Generate a list of key features based on file summaries.

    Parameters:
    combined_summary (str): A string containing the combined summaries of the project files.
    feature_component (FeatureQA): The component used to extract key features.

    Returns:
    str: The generated list of key features in the specified format.
    """
    return feature_component.generator.call({"input_str": combined_summary})

In [29]:
# Initialize FeatureQA component with model configuration
feature_qa = FeatureQA(**model)

if combined_summary:
    # Generate the key features based on the combined summary
    key_feature = generate_key_feature(summary, feature_component=feature_qa)

    # Process the key features to ensure proper formatting
    key_feature = get_description_data(key_feature)

    # Create markdown format for key features
    key_feature_markdown = "# Key Features\n" + key_feature

    # Print the key features in markdown format
    print(key_feature_markdown)

    # Display the key features in a Markdown view (for environments that support it)
    display(Markdown(key_feature_markdown))

# Key Features
* **1. Authenticates with Bright Data proxy using username and password**: Utilizes a proxy service for secure and reliable web scraping.
* **2. Retrieves HTML content from the provided URL**: Extracts webpage data using Axios, enabling access to dynamic web pages.
* **3. Extracts product information from the script tags in the HTML using Cheerio**: Parses HTML to gather product details, streamlining data collection.
* **4. Parses JSON data to extract various product attributes**: Converts JSON data into structured product info, facilitating easier analysis and use cases.
* **5. Creates a Product object with the extracted information**: Organizes scraped data into a unified Product object, promoting efficient storage and processing.
* **6. Returns the Product object or throws an error if scraping fails**: Provides a clear outcome for successful or failed web scraping attempts.
* **7. Extracted Product Attributes**: Includes essential product details such as title, descri

# Key Features
* **1. Authenticates with Bright Data proxy using username and password**: Utilizes a proxy service for secure and reliable web scraping.
* **2. Retrieves HTML content from the provided URL**: Extracts webpage data using Axios, enabling access to dynamic web pages.
* **3. Extracts product information from the script tags in the HTML using Cheerio**: Parses HTML to gather product details, streamlining data collection.
* **4. Parses JSON data to extract various product attributes**: Converts JSON data into structured product info, facilitating easier analysis and use cases.
* **5. Creates a Product object with the extracted information**: Organizes scraped data into a unified Product object, promoting efficient storage and processing.
* **6. Returns the Product object or throws an error if scraping fails**: Provides a clear outcome for successful or failed web scraping attempts.
* **7. Extracted Product Attributes**: Includes essential product details such as title, description, price, ratings, and more, enabling comprehensive understanding of e-commerce data.

<!-- @format -->

### Getting Started


In [30]:
from lightrag.core.generator import Generator
from lightrag.core.component import Component
from lightrag.components.model_client import OllamaClient

# Define the template for generating installation instructions
installation_template = r"""<SYS>
You are a highly skilled software engineer with expertise in documentation and project setup. You are adept at analyzing project summaries and folder structures to create clear installation instructions.
</SYS>
Based on the following project summary and folder structure:
Summary:
{{project_summary}}

Folder Structure:
{{folder_structure}}

GitHub repo Link:
{{repo_link}}

Create a detailed installation guide that includes:
1. **Prerequisites**: List any software, tools, or environment setups required before installation (e.g., Node.js, Docker) and provide link to download or install them.
2. **Setup Instructions**: Step-by-step instructions to set up the project, including installing dependencies, configuring environment variables, and any other necessary setup.
3. **Running the Project**: Detailed commands and steps to run the project locally, including any necessary build steps or configuration commands.
4. **Troubleshooting**: Common issues that may arise during installation and how to resolve them.
Ensure that the installation guide is comprehensive and easy to follow for someone new to the project and properly format them with heading like # Getting Started, ## Prerequisites, ## Installation, ## Running the Project, ## Tests, ## Troubleshooting
Summary:"""


class InstallationQA(Component):
    """Component for generating installation instructions."""

    def __init__(self, model_client: OllamaClient, model_kwargs: dict):  # type: ignore
        """
        Initialize the InstallationQA component.

        Parameters:
        - model_client (OllamaClient): The client to interact with the model.
        - model_kwargs (dict): Additional arguments for the model.
        """
        super().__init__()
        # Create a Generator instance with the provided model client and template
        self.generator = Generator(
            model_client=model_client,
            model_kwargs=model_kwargs,
            template=installation_template,
        )

    def call(self, project_summary: str, folder_structure: str, repo_link: str) -> str:
        """
        Generate installation guide based on project summary, folder structure, and repo link.

        Parameters:
        - project_summary (str): Summary of the project.
        - folder_structure (str): Structure of the project folders.
        - repo_link (str): URL of the GitHub repository.

        Returns:
        - str: The generated installation guide.
        """
        return self.generator.call(
            {
                "project_summary": project_summary,
                "folder_structure": folder_structure,
                "repo_link": repo_link,
            }
        )

    async def acall(
        self, project_summary: str, folder_structure: str, repo_link: str
    ) -> str:
        """
        Asynchronous version for generating installation guide.

        Parameters:
        - project_summary (str): Summary of the project.
        - folder_structure (str): Structure of the project folders.
        - repo_link (str): URL of the GitHub repository.

        Returns:
        - str: The generated installation guide.
        """
        return await self.generator.acall(
            {
                "project_summary": project_summary,
                "folder_structure": folder_structure,
                "repo_link": repo_link,
            }
        )

In [31]:
def generate_installation_guide(
    project_summary: str,
    folder_structure: str,
    repo_link: str,
    installation_component: InstallationQA,
) -> str:
    """
    Generate a comprehensive installation guide for the project.

    Parameters:
    - project_summary (str): Summary of the project.
    - folder_structure (str): Structure of the project folders.
    - repo_link (str): URL of the GitHub repository.
    - installation_component (InstallationQA): Component for generating the installation guide.

    Returns:
    - str: The generated installation guide.
    """
    return installation_component.call(project_summary, folder_structure, repo_link)


# Initialize InstallationQA component with model configuration
installation_qa = InstallationQA(**model)

if combined_summary:
    # Generate the installation guide using the provided summaries and folder structure
    installation_guide = generate_installation_guide(
        combined_summary,
        folder_structure_str,
        repository_url,
        installation_component=installation_qa,
    )
    # Process the generated guide (e.g., clean or format the content)
    installation_guide = get_description_data(installation_guide)

    # Format and display the installation guide in Markdown
    installation_guide_markdown = installation_guide
    print(installation_guide_markdown)
    display(Markdown(installation_guide_markdown))

Here's a detailed installation guide:

**Getting Started**

Welcome to the Daraz Scraper installation guide! This guide will walk you through the process of setting up the project on your local machine.

**Prerequisites**

Before installing the project, make sure you have the following software and tools installed:

* Node.js (LTS version) - [Download](https://nodejs.org/en/download/)
* yarn (package manager) - [Install with npm](https://yarnpkg.com/en/docs/install)
* Docker (for testing purposes) - [Download](https://www.docker.com/get-started)
* MongoDB (database) - [Download](https://www.mongodb.com/try/download/community)

**Setup Instructions**

### Step 1: Clone the Repository

Clone the Daraz Scraper repository from GitHub using the following command:
```bash
git clone https://github.com/Eemayas/Daraz_Scraper.git
```
### Step 2: Install Dependencies

Navigate to the project directory and install dependencies using yarn:
```bash
cd Daraz_Scraper
yarn install
```
This will take a 

Here's a detailed installation guide:

**Getting Started**
================

Welcome to the Daraz Scraper installation guide! This guide will walk you through the process of setting up the project on your local machine.

**Prerequisites**
================

Before installing the project, make sure you have the following software and tools installed:

* Node.js (LTS version) - [Download](https://nodejs.org/en/download/)
* yarn (package manager) - [Install with npm](https://yarnpkg.com/en/docs/install)
* Docker (for testing purposes) - [Download](https://www.docker.com/get-started)
* MongoDB (database) - [Download](https://www.mongodb.com/try/download/community)

**Setup Instructions**
=====================

### Step 1: Clone the Repository

Clone the Daraz Scraper repository from GitHub using the following command:
```bash
git clone https://github.com/Eemayas/Daraz_Scraper.git
```
### Step 2: Install Dependencies

Navigate to the project directory and install dependencies using yarn:
```bash
cd Daraz_Scraper
yarn install
```
This will take a few minutes to complete.

### Step 3: Configure Environment Variables

Create a new file called `.env` in the root directory of the project. Add the following environment variables:
```makefile
MONGO_URI=mongodb://localhost:27017/
API_KEY=your_api_key_here
```
Replace `your_api_key_here` with your actual API key.

### Step 4: Build and Start the Project

Run the following command to build and start the project:
```bash
yarn dev
```
This will start the development server, which you can access at [http://localhost:3000](http://localhost:3000).

**Running the Project**
=====================

To run the project in production mode, use the following command:
```bash
yarn build
yarn start
```
This will create a static HTML file and serve it using `next.js`.

**Tests**
========

The project uses Jest for testing. To run tests, navigate to the root directory of the project and run the following command:
```bash
yarn test
```

**Troubleshooting**
==================

### Common Issues

* **Error: Cannot find module 'node_modules'**: Make sure you have installed dependencies using yarn.
* **Error: MongoDB connection failed**: Check your MongoDB installation and ensure that it is running on the correct port (27017).
* **Error: API key not found**: Verify that you have added the API key to the `.env` file.

### Resolving Issues

If you encounter any issues during installation or while running the project, refer to the following troubleshooting steps:

* Run `yarn install` again to re-install dependencies.
* Restart MongoDB and ensure it is running on the correct port (27017).
* Verify that you have added the API key to the `.env` file.

Summary
--------

Congratulations! You have successfully installed the Daraz Scraper project. If you encounter any issues during installation or while running the project, refer to this guide for troubleshooting steps.

<!-- @format -->

### API Reference


In [32]:
from lightrag.core.generator import Generator
from lightrag.core.component import Component


# Define the template for API reference extraction
api_template_v1 = r"""<SYS>
You are an API reference extraction assistant specialized in coding files.
</SYS>
Please extract the API reference from the following code and provide the following information:
1. API endpoint
2. Purpose of the API
3. Parameters
4. Parameter types
5. Parameter descriptions
6. HTTP method

Format:

#### {Purpose of the API}

```http
  {HTTP method} {API endpoint}
```

| Parameter | Type     | Description                |
| :-------- | :------- | :------------------------- |
{parameter_rows}

Example:

#### Get all items

```http
  GET /api/items
```

| Parameter | Type     | Description                |
| :-------- | :------- | :------------------------- |
| `api_key` | `string` | **Required**. Your API key |
| `limit`   | `integer`| **Optional**. Limit the number of items |


If there is no API Reference, please return "No API Reference". No description needed in that case. Avoid asking for response like this "Let me know if you'd like me to clarify anything!" and write the Notes in third person narrative

Code:
{{input_str}}
"""

# Define the template for API reference extraction
api_template = r"""
You are an API reference extraction assistant specializing in coding files. Your task is to identify and extract information about HTTP API methods from the provided code. Focus only on endpoints that use HTTP methods (GET, POST, PUT, DELETE, etc.).

For each API reference found, provide the following details:
1. API endpoint
2. Purpose of the API
3. Parameters
4. Parameter types
5. Parameter descriptions
6. HTTP method

If the code does not include any API references or if no HTTP methods are present, return "No API Reference."

Format:

#### {Purpose of the API}

```http
  {HTTP method} {API endpoint}
```

| Parameter | Type     | Description                |
| :-------- | :------- | :------------------------- |
{parameter_rows}

Example:

#### Get all items

```http
  GET /api/items
```

| Parameter | Type     | Description                |
| :-------- | :------- | :------------------------- |
| `api_key` | `string` | **Required**. Your API key |
| `limit`   | `integer`| **Optional**. Limit the number of items |

Code:
{{input_str}}

"""


class APIReferenceExtractorQA(Component):
    """Component for extracting API references from code."""

    def __init__(self, model_client: OllamaClient, model_kwargs: dict):  # type: ignore
        super().__init__()
        self.generator = Generator(
            model_client=model_client,
            model_kwargs=model_kwargs,
            template=api_template,
        )

    def call(self, input: str) -> str:
        """Extract API references from the provided code."""
        return self.generator.call({"input_str": input})

    async def acall(self, input: str) -> str:
        """Asynchronous extraction of API references."""
        return await self.generator.acall({"input_str": input})

In [33]:
import os
from pathlib import Path
from typing import List, Dict
from tqdm import tqdm
from prettytable import PrettyTable


def generate_api_reference(
    path: Path,
    ignore_list: List[str],
    api_reference_extractor_component: APIReferenceExtractorQA,
    api_ignore_extensions: List[str],
) -> List[Dict[str, str]]:
    """
    Generate an API reference of files in the given path using the model.

    This function processes files in the specified directory, excluding those
    listed in the ignore list and files with extensions specified in
    `api_ignore_extensions`. It uses the `APIReferenceExtractorQA` component
    to extract API references from the file content.

    Parameters:
    path (Path): The directory path containing the files to process.
    ignore_list (List[str]): List of folder names or file names to ignore.
    api_reference_extractor_component (APIReferenceExtractorQA): The component used to extract API references.
    api_ignore_extensions (List[str]): List of file extensions to ignore.

    Returns:
    List[Dict[str, str]]: A list of dictionaries where each dictionary contains
    the file path and the extracted API reference or an error message.
    """
    api_reference = []
    files_to_process = []

    for root, dirs, files in os.walk(path):
        relative_root = os.path.relpath(root, path)

        # Skip directories that are in the ignore list
        if any(ignored in relative_root.split(os.sep) for ignored in ignore_list):
            continue

        for file in files:
            file_path = Path(root) / file

            # Skip files that are in the ignore list or have ignored extensions
            if any(ignored in file_path.parts for ignored in ignore_list):
                continue

            if any(ignore in file_path.name for ignore in specific_ignores_api):
                continue

            if file_path.suffix.lower() in api_ignore_extensions:
                continue

            files_to_process.append(file_path)

    # Create a progress bar for processing files
    pbar = tqdm(files_to_process, unit="file")
    for file_path in pbar:
        # Update the progress bar description
        pbar.set_description(f"Processing files - {file_path}")
        try:
            with open(file_path, "r") as f:
                file_content = f.read()

            # Extract API references from the file content
            api_text = api_reference_extractor_component.call(file_content)
            api_reference.append({"file": file_path, "api_reference": api_text})
        except Exception as e:
            # Append an error message if file processing fails
            api_reference.append(
                {"file": file_path, "api_reference": f"Error processing file: {str(e)}"}
            )

    return api_reference


# Initialize APIReferenceExtractorQA component with model configuration
api_reference_extractor_qa = APIReferenceExtractorQA(**model)

if cloned_repo_path:
    path = Path(cloned_repo_path)
    if not path.is_dir():
        print(f"The path {path} is not a directory.")
    api_reference = generate_api_reference(
        path,
        ignore_list=ignore_list_folder_structure,
        api_reference_extractor_component=api_reference_extractor_qa,
        api_ignore_extensions=api_ignore_extensions,
    )

    # Initialize PrettyTable for formatted output
    api_table = PrettyTable()
    api_table.field_names = ["File", "API Reference"]

    # Create a list to hold data for further processing (e.g., saving to Excel)
    api_data_for_excel = []

    for item in api_reference:
        # Process the API reference text to get description data
        description_data = get_description_data(item["api_reference"])
        if "No API Reference" in description_data:
            continue
        # Add rows to the PrettyTable
        api_table.add_row([item["file"], description_data])
        api_data_for_excel.append(
            {"File": item["file"], "api_reference": description_data}
        )

    # Print the PrettyTable
    print(api_table)
else:
    print("Repository cloning failed or was skipped.")

Processing files - Github_repos\Daraz_Scraper\types\index.ts: 100%|██████████| 17/17 [00:36<00:00,  2.15s/file]             

+--------------------------------------------------+---------------------------------------------------------------------------------------------------------+
|                       File                       |                                              API Reference                                              |
+--------------------------------------------------+---------------------------------------------------------------------------------------------------------+
| Github_repos\Daraz_Scraper\app\api\cron\route.ts |                                          #### Get all products                                          |
|                                                  |                                                                                                         |
|                                                  |                                                 ```http                                                 |
|                                             

In [34]:
for item in api_reference:
    # Check if the item is a dictionary and contains the 'api_reference' key
    if isinstance(item, dict) and "api_reference" in item:
        description_data = get_description_data(item["api_reference"])

        # Skip items with no API reference
        if "No API Reference" in description_data:
            continue

        # Add valid items to the PrettyTable
        api_table.add_row([item["file"], description_data])
        api_data_for_excel.append(
            {"File": item["file"], "api_reference": description_data}
        )
    else:
        # Print a message if the item format is unexpected
        print(f"Unexpected item format: {item}\n")

<!-- @format -->

#### API Reference Data Export


In [46]:
import pandas as pd

if api_data_for_excel:
    # Convert the list of dictionaries to a DataFrame
    df_api_data = pd.DataFrame(api_data_for_excel)

    # Define the path and name for the Excel file
    excel_path = f"output/{metadata.name}_api_reference.xlsx"

    # Save the DataFrame to an Excel file with the specified path
    df_api_data.to_excel(excel_path, index=False, engine="openpyxl")

    # Print a confirmation message with the file path
    print(f"API reference data saved to {excel_path}")

API reference data saved to output/Daraz_Scraper_api_reference.xlsx


<!-- @format -->

#### API Reference Markdown


In [36]:
# Convert to string in the desired format
api_reference_markdown = "# API Reference\n\n"
for i, entry in enumerate(api_data_for_excel, start=1):
    file = entry["File"]
    api_reference = entry["api_reference"]
    # Append formatted API reference to the markdown string
    api_reference_markdown += f"**File:** {file}\n\n{api_reference}\n\n"

print(api_reference_markdown)
display(Markdown(api_reference_markdown))

# API Reference

**File:** Github_repos\Daraz_Scraper\app\api\cron\route.ts

#### Get all products

```http
  GET /api/products
```

| Parameter | Type     | Description                |
| :-------- | :------- | :------------------------- |
| None      | -        | No parameters required for this API endpoint |

Note: There are no other HTTP methods (e.g., POST, PUT, DELETE) mentioned in the provided code snippet.

**File:** Github_repos\Daraz_Scraper\app\api\cron\route.ts

#### Get all products

```http
  GET /api/products
```

| Parameter | Type     | Description                |
| :-------- | :------- | :------------------------- |
| None      | -        | No parameters required for this API endpoint |

Note: There are no other HTTP methods (e.g., POST, PUT, DELETE) mentioned in the provided code snippet.




# API Reference

**File:** Github_repos\Daraz_Scraper\app\api\cron\route.ts

#### Get all products

```http
  GET /api/products
```

| Parameter | Type     | Description                |
| :-------- | :------- | :------------------------- |
| None      | -        | No parameters required for this API endpoint |

Note: There are no other HTTP methods (e.g., POST, PUT, DELETE) mentioned in the provided code snippet.

**File:** Github_repos\Daraz_Scraper\app\api\cron\route.ts

#### Get all products

```http
  GET /api/products
```

| Parameter | Type     | Description                |
| :-------- | :------- | :------------------------- |
| None      | -        | No parameters required for this API endpoint |

Note: There are no other HTTP methods (e.g., POST, PUT, DELETE) mentioned in the provided code snippet.



<!-- @format -->

### Contributing


In [37]:
def generate_contributing_guide(repo_link):
    """
    Generate a contributing guide for a GitHub repository.

    Parameters:
    repo_link (str): The URL of the GitHub repository.

    Returns:
    str: The contributing guide in markdown format.
    """
    import re

    # Extract the username and repository name from the link
    match = re.match(r"https://github.com/([^/]+)/([^/]+)", repo_link)
    if not match:
        raise ValueError("Invalid GitHub repository link")

    username, repo_name = match.groups()

    # Define the guide with placeholders for URLs
    guide_template = f"""
# Contributing

Contributions are welcome! Here are several ways you can contribute:

- **[Submit Pull Requests](https://github.com/{username}/{repo_name}/pulls)**: Review open PRs, and submit your own PRs.
- **[Join the Discussions](https://github.com/{username}/{repo_name}/discussions)**: Share your insights, provide feedback, or ask questions.
- **[Report Issues](https://github.com/{username}/{repo_name}/issues)**: Submit bugs found or log feature requests for {repo_name}.

### Contributing Guidelines

1. **Fork the Repository**:
    - Start by forking the project repository to your GitHub account.
2. **Clone the Repository**:
    - Clone your forked repository to your local machine using the command:
    ```sh
    git clone https://github.com/your-username/{repo_name}.git
    ```
    - Replace ``your-username`` with your GitHub username.
3. **Create a New Branch**:
    - Create a new branch for your changes using the command:
    ```sh
    git checkout -b your-branch-name
    ```
4. **Make Your Changes**:
    - Edit, add, or delete files as needed. Ensure your changes align with the project's contribution guidelines.
5. **Commit Your Changes**:
    - Stage your changes and commit them with a descriptive message:
      ```bash
      git add .
      git commit -m "Your descriptive message"
      ```
6. **Push Your Changes:**
    - Push your branch to your forked repository:
      ```bash
      git push origin your-branch-name
      ```
7. **Create a Pull Request (PR):**
    - Go to the original repository on GitHub and click “Compare & pull request.” Provide a clear description of the changes and submit the PR.

Once your PR is reviewed and approved, it will be merged into the main branch.
    """

    return guide_template


# Generate the contributing guide for the given repository URL
contribution_markdown = generate_contributing_guide(repository_url)
print(contribution_markdown)
display(Markdown(contribution_markdown))


# Contributing

Contributions are welcome! Here are several ways you can contribute:

- **[Submit Pull Requests](https://github.com/Eemayas/Daraz_Scraper/pulls)**: Review open PRs, and submit your own PRs.
- **[Join the Discussions](https://github.com/Eemayas/Daraz_Scraper/discussions)**: Share your insights, provide feedback, or ask questions.
- **[Report Issues](https://github.com/Eemayas/Daraz_Scraper/issues)**: Submit bugs found or log feature requests for Daraz_Scraper.

### Contributing Guidelines

1. **Fork the Repository**:
    - Start by forking the project repository to your GitHub account.
2. **Clone the Repository**:
    - Clone your forked repository to your local machine using the command:
    ```sh
    git clone https://github.com/your-username/Daraz_Scraper.git
    ```
    - Replace ``your-username`` with your GitHub username.
3. **Create a New Branch**:
    - Create a new branch for your changes using the command:
    ```sh
    git checkout -b your-branch-name
    ```


# Contributing

Contributions are welcome! Here are several ways you can contribute:

- **[Submit Pull Requests](https://github.com/Eemayas/Daraz_Scraper/pulls)**: Review open PRs, and submit your own PRs.
- **[Join the Discussions](https://github.com/Eemayas/Daraz_Scraper/discussions)**: Share your insights, provide feedback, or ask questions.
- **[Report Issues](https://github.com/Eemayas/Daraz_Scraper/issues)**: Submit bugs found or log feature requests for Daraz_Scraper.

### Contributing Guidelines

1. **Fork the Repository**:
    - Start by forking the project repository to your GitHub account.
2. **Clone the Repository**:
    - Clone your forked repository to your local machine using the command:
    ```sh
    git clone https://github.com/your-username/Daraz_Scraper.git
    ```
    - Replace ``your-username`` with your GitHub username.
3. **Create a New Branch**:
    - Create a new branch for your changes using the command:
    ```sh
    git checkout -b your-branch-name
    ```
4. **Make Your Changes**:
    - Edit, add, or delete files as needed. Ensure your changes align with the project's contribution guidelines.
5. **Commit Your Changes**:
    - Stage your changes and commit them with a descriptive message:
      ```bash
      git add .
      git commit -m "Your descriptive message"
      ```
6. **Push Your Changes:**
    - Push your branch to your forked repository:
      ```bash
      git push origin your-branch-name
      ```
7. **Create a Pull Request (PR):**
    - Go to the original repository on GitHub and click “Compare & pull request.” Provide a clear description of the changes and submit the PR.

Once your PR is reviewed and approved, it will be merged into the main branch.
    

<!-- @format -->

### Contributors


In [38]:
def generate_contributors_table(contributors):
    """
    Generate a markdown table of contributors with their avatars, GitHub profiles, and contribution counts.

    Parameters:
    contributors (list): A list of contributor objects with attributes 'avatar_url', 'name', 'profile_url', and 'contributions'.

    Returns:
    str: The contributors table in markdown format.
    """
    # Start with the table header
    table = "| Avatar | Contributor | GitHub Profile | No of Contributions |\n"
    table += "|:--------:|:--------------:|:----------------:|:-------------------:|\n"

    # Add each contributor to the table
    for contributor in contributors:
        table += (
            f"| <img src='{contributor.avatar_url}' width='40' height='40' style='border-radius:50%;'/> | "
            f"{contributor.name} | "
            f"[@{contributor.name}]({contributor.profile_url}) | "
            f"{contributor.contributions} |\n"
        )

    return table


# Example usage for generating contributors markdown
contributor_markdown = f"""
# Contributors\n
{generate_contributors_table(metadata.contributors)}
    """
print(contributor_markdown)
display(Markdown(contributor_markdown))


# Contributors

| Avatar | Contributor | GitHub Profile | No of Contributions |
|:--------:|:--------------:|:----------------:|:-------------------:|
| <img src='https://avatars.githubusercontent.com/u/100434825?v=4' width='40' height='40' style='border-radius:50%;'/> | Eemayas | [@Eemayas](https://github.com/Eemayas) | 14 |

    



# Contributors

| Avatar | Contributor | GitHub Profile | No of Contributions |
|:--------:|:--------------:|:----------------:|:-------------------:|
| <img src='https://avatars.githubusercontent.com/u/100434825?v=4' width='40' height='40' style='border-radius:50%;'/> | Eemayas | [@Eemayas](https://github.com/Eemayas) | 14 |

    

<!-- @format -->

### License


In [39]:
license_markdown = """
# License

This project is licensed under the MIT License - see the [LICENSE](./LICENSE) file for details.
"""

# Display the license information
print(license_markdown)
display(Markdown(license_markdown))


# License

This project is licensed under the MIT License - see the [LICENSE](./LICENSE) file for details.




# License

This project is licensed under the MIT License - see the [LICENSE](./LICENSE) file for details.


<!-- @format -->

# Markdown


<!-- @format -->

### Combine and Save README Content


In [40]:
# Combine all markdown sections into a single markdown string
combined_markdown = (
    header_markdown
    + "\n\n---\n"
    + project_overview_markdown
    + "\n\n---\n"
    + key_feature_markdown
    + "\n\n---\n"
    + folder_structure_markdown
    + "\n\n---\n"
    + installation_guide_markdown
    + "\n\n---\n"
    + api_reference_markdown
    + "\n\n---\n"
    + contribution_markdown
    + "\n\n---\n"
    + contributor_markdown
    + "\n\n---\n"
    + license_markdown
    + "\n\n---\n"
)

# Display the combined markdown
print(combined_markdown)
display(Markdown(combined_markdown))


<p align="center">
    <img src="https://img.icons8.com/nolan/512/1A6DFF/C822FF/shopping-basket-2.png" width="200" style="border-radius: 20px;" />
</p>
    

<p align="center">
    <h1>Daraz_Scraper</h1>
</p>


<p align="center">
  <img src="https://img.shields.io/github/license/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="license">
  <img src="https://img.shields.io/github/last-commit/Eemayas/Daraz_Scraper?style=flat&logo=git&logoColor=white&color=0080ff" alt="last-commit">
  <img src="https://img.shields.io/github/languages/top/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="repo-top-language">
  <img src="https://img.shields.io/github/languages/count/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="repo-language-count">
  <img src="https://img.shields.io/github/actions/workflow/status/Eemayas/Daraz_Scraper/build.yml?branch=main&style=flat&color=0080ff" alt="build-status">
  <img src="https://img.shields.io/github/issues/Eemayas/Daraz_Scraper?style=flat&color=0080ff" a


<p align="center">
    <img src="https://img.icons8.com/nolan/512/1A6DFF/C822FF/shopping-basket-2.png" width="200" style="border-radius: 20px;" />
</p>
    

<p align="center">
    <h1>Daraz_Scraper</h1>
</p>


<p align="center">
  <img src="https://img.shields.io/github/license/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="license">
  <img src="https://img.shields.io/github/last-commit/Eemayas/Daraz_Scraper?style=flat&logo=git&logoColor=white&color=0080ff" alt="last-commit">
  <img src="https://img.shields.io/github/languages/top/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="repo-top-language">
  <img src="https://img.shields.io/github/languages/count/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="repo-language-count">
  <img src="https://img.shields.io/github/actions/workflow/status/Eemayas/Daraz_Scraper/build.yml?branch=main&style=flat&color=0080ff" alt="build-status">
  <img src="https://img.shields.io/github/issues/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="open-issues">
  <img src="https://img.shields.io/github/forks/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="forks">
  <img src="https://img.shields.io/github/stars/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="stars">
  <img src="https://img.shields.io/github/issues-pr/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="pull-requests">
  <img src="https://img.shields.io/github/contributors/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="contributors">
  <img src="https://img.shields.io/github/commit-activity/m/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="commit-activity">
  <img src="https://img.shields.io/github/languages/code-size/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="code-size">
  <img src="https://img.shields.io/github/repo-size/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="repo-size">
  <img src="https://img.shields.io/github/downloads/Eemayas/Daraz_Scraper/total?style=flat&color=0080ff" alt="downloads">
  <img src="https://img.shields.io/github/sponsors/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="sponsors">
  <img src="https://img.shields.io/github/v/release/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="release-version">
  <img src="https://img.shields.io/codecov/c/github/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="coverage">
  <img src="https://img.shields.io/codeclimate/quality/a/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="code-quality">
  <img src="https://img.shields.io/david/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="dependencies">
  <img src="https://img.shields.io/david/dev/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="dev-dependencies">
  <img src="https://img.shields.io/snyk/vulnerabilities/github/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="security">
  <img src="https://img.shields.io/website?style=flat&color=0080ff&url=https%3A%2F%2Fexample.com" alt="performance">
  <img src="https://img.shields.io/github/commit-activity/y/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="activity">
  <img src="https://img.shields.io/docsify/docs?style=flat&color=0080ff" alt="documentation">
  <img src="https://img.shields.io/github/v/tag/Eemayas/Daraz_Scraper?style=flat&color=0080ff" alt="version">
</p>
    

<p align="center">
    <em>Constructed using the following tools and technologies:</em>
</p>

<p align="center">
  <img src="https://img.shields.io/badge/CSS3-1572B6.svg?style=for-the-badge&logo=CSS3&logoColor=white" alt="CSS">
  <img src="https://img.shields.io/badge/CSS3-1572B6.svg?style=for-the-badge&logo=CSS3&logoColor=white" alt="CSS3">
  <img src="https://img.shields.io/badge/TypeScript-3178C6.svg?style=for-the-badge&logo=TypeScript&logoColor=white" alt="TypeScript">
  <img src="https://img.shields.io/badge/HTML5-E34F26.svg?style=for-the-badge&logo=HTML5&logoColor=white" alt="HTML5">
  <img src="https://img.shields.io/badge/JavaScript-F7DF1E.svg?style=for-the-badge&logo=JavaScript&logoColor=black" alt="JavaScript">
  <img src="https://img.shields.io/badge/Electron-47848F.svg?style=for-the-badge&logo=Electron&logoColor=white" alt="Electron">
  <img src="https://img.shields.io/badge/Next.js-000000.svg?style=for-the-badge&logo=nextdotjs&logoColor=white" alt="Next.js">
</p>
    



---
# Project Overview

The project is an e-commerce platform that enables users to buy and sell products, with a focus on product pricing, reviews, and stock management. It allows sellers to list their products with detailed information, such as price history, discount rates, and seller ratings, while also enabling buyers to review and rate the products they purchase. The project provides features like automatic email notifications for new orders, low stock levels, and threshold met, allowing users to stay informed about important events related to their business or purchases.

The project uses Mongoose to interact with a MongoDB database, storing products in a collection called "products". It also utilizes Node.js, Axios, Cheerio, and Nodemailer libraries to handle product scraping from Daraz website, email generation, and sending emails based on notification types. The key features of the project include automated email notifications for various events, product pricing and review management, stock level tracking, and seller information display.

---
# Key Features
* **1. Authenticates with Bright Data proxy using username and password**: Utilizes a proxy service for secure and reliable web scraping.
* **2. Retrieves HTML content from the provided URL**: Extracts webpage data using Axios, enabling access to dynamic web pages.
* **3. Extracts product information from the script tags in the HTML using Cheerio**: Parses HTML to gather product details, streamlining data collection.
* **4. Parses JSON data to extract various product attributes**: Converts JSON data into structured product info, facilitating easier analysis and use cases.
* **5. Creates a Product object with the extracted information**: Organizes scraped data into a unified Product object, promoting efficient storage and processing.
* **6. Returns the Product object or throws an error if scraping fails**: Provides a clear outcome for successful or failed web scraping attempts.
* **7. Extracted Product Attributes**: Includes essential product details such as title, description, price, ratings, and more, enabling comprehensive understanding of e-commerce data.

---
# Folder Structure
```sh
Daraz_Scraper/
├── .env.local.copy
├── .gitignore
├── app/
│   ├── api/
│   │   └── cron/
│   │       └── route.ts
│   ├── favicon.ico
│   ├── globals.css
│   ├── layout.tsx
│   ├── page.tsx
│   └── products/
│       └── [id]/
│           └── page.tsx
├── components/
│   ├── HeroCarousel.tsx
│   ├── Modal.tsx
│   ├── Navbar.tsx
│   ├── PriceInfoCard.tsx
│   ├── ProductCard.tsx
│   └── Searchbar.tsx
├── lib/
│   ├── action/
│   │   └── index.ts
│   ├── models/
│   │   └── product.model.ts
│   ├── mongoose.ts
│   ├── nodemailer/
│   │   └── index.ts
│   ├── scrapper/
│   │   └── index.ts
│   └── utils.ts
├── next.config.js
├── package-lock.json
├── package.json
├── postcss.config.js
├── public/
│   ├── assets/
│   │   ├── icons/
│   │   │   ├── arrow-down.svg
│   │   │   ├── arrow-right.svg
│   │   │   ├── arrow-up.svg
│   │   │   ├── bag.svg
│   │   │   ├── black-heart.svg
│   │   │   ├── bookmark.svg
│   │   │   ├── chart.svg
│   │   │   ├── check.svg
│   │   │   ├── chevron-down.svg
│   │   │   ├── comment.svg
│   │   │   ├── frame.svg
│   │   │   ├── hand-drawn-arrow.svg
│   │   │   ├── logo.svg
│   │   │   ├── mail.svg
│   │   │   ├── price-tag.svg
│   │   │   ├── red-heart.svg
│   │   │   ├── search.svg
│   │   │   ├── share.svg
│   │   │   ├── square.svg
│   │   │   ├── star.svg
│   │   │   ├── user.svg
│   │   │   └── x-close.svg
│   │   └── images/
│   │       ├── details.svg
│   │       ├── hero-1.svg
│   │       ├── hero-2.svg
│   │       ├── hero-3.svg
│   │       ├── hero-4.svg
│   │       ├── hero-5.svg
│   │       └── trending.svg
│   ├── demo/
│   │   ├── home-page.png
│   │   ├── product-page.png
│   │   └── track-product.png
│   ├── next.svg
│   └── vercel.svg
├── README.md
├── tailwind.config.ts
├── tsconfig.json
├── types/
│   └── index.ts
└── vercel.json

17 directories, 63 files
```

---
Here's a detailed installation guide:

**Getting Started**
================

Welcome to the Daraz Scraper installation guide! This guide will walk you through the process of setting up the project on your local machine.

**Prerequisites**
================

Before installing the project, make sure you have the following software and tools installed:

* Node.js (LTS version) - [Download](https://nodejs.org/en/download/)
* yarn (package manager) - [Install with npm](https://yarnpkg.com/en/docs/install)
* Docker (for testing purposes) - [Download](https://www.docker.com/get-started)
* MongoDB (database) - [Download](https://www.mongodb.com/try/download/community)

**Setup Instructions**
=====================

### Step 1: Clone the Repository

Clone the Daraz Scraper repository from GitHub using the following command:
```bash
git clone https://github.com/Eemayas/Daraz_Scraper.git
```
### Step 2: Install Dependencies

Navigate to the project directory and install dependencies using yarn:
```bash
cd Daraz_Scraper
yarn install
```
This will take a few minutes to complete.

### Step 3: Configure Environment Variables

Create a new file called `.env` in the root directory of the project. Add the following environment variables:
```makefile
MONGO_URI=mongodb://localhost:27017/
API_KEY=your_api_key_here
```
Replace `your_api_key_here` with your actual API key.

### Step 4: Build and Start the Project

Run the following command to build and start the project:
```bash
yarn dev
```
This will start the development server, which you can access at [http://localhost:3000](http://localhost:3000).

**Running the Project**
=====================

To run the project in production mode, use the following command:
```bash
yarn build
yarn start
```
This will create a static HTML file and serve it using `next.js`.

**Tests**
========

The project uses Jest for testing. To run tests, navigate to the root directory of the project and run the following command:
```bash
yarn test
```

**Troubleshooting**
==================

### Common Issues

* **Error: Cannot find module 'node_modules'**: Make sure you have installed dependencies using yarn.
* **Error: MongoDB connection failed**: Check your MongoDB installation and ensure that it is running on the correct port (27017).
* **Error: API key not found**: Verify that you have added the API key to the `.env` file.

### Resolving Issues

If you encounter any issues during installation or while running the project, refer to the following troubleshooting steps:

* Run `yarn install` again to re-install dependencies.
* Restart MongoDB and ensure it is running on the correct port (27017).
* Verify that you have added the API key to the `.env` file.

Summary
--------

Congratulations! You have successfully installed the Daraz Scraper project. If you encounter any issues during installation or while running the project, refer to this guide for troubleshooting steps.

---
# API Reference

**File:** Github_repos\Daraz_Scraper\app\api\cron\route.ts

#### Get all products

```http
  GET /api/products
```

| Parameter | Type     | Description                |
| :-------- | :------- | :------------------------- |
| None      | -        | No parameters required for this API endpoint |

Note: There are no other HTTP methods (e.g., POST, PUT, DELETE) mentioned in the provided code snippet.

**File:** Github_repos\Daraz_Scraper\app\api\cron\route.ts

#### Get all products

```http
  GET /api/products
```

| Parameter | Type     | Description                |
| :-------- | :------- | :------------------------- |
| None      | -        | No parameters required for this API endpoint |

Note: There are no other HTTP methods (e.g., POST, PUT, DELETE) mentioned in the provided code snippet.



---

# Contributing

Contributions are welcome! Here are several ways you can contribute:

- **[Submit Pull Requests](https://github.com/Eemayas/Daraz_Scraper/pulls)**: Review open PRs, and submit your own PRs.
- **[Join the Discussions](https://github.com/Eemayas/Daraz_Scraper/discussions)**: Share your insights, provide feedback, or ask questions.
- **[Report Issues](https://github.com/Eemayas/Daraz_Scraper/issues)**: Submit bugs found or log feature requests for Daraz_Scraper.

### Contributing Guidelines

1. **Fork the Repository**:
    - Start by forking the project repository to your GitHub account.
2. **Clone the Repository**:
    - Clone your forked repository to your local machine using the command:
    ```sh
    git clone https://github.com/your-username/Daraz_Scraper.git
    ```
    - Replace ``your-username`` with your GitHub username.
3. **Create a New Branch**:
    - Create a new branch for your changes using the command:
    ```sh
    git checkout -b your-branch-name
    ```
4. **Make Your Changes**:
    - Edit, add, or delete files as needed. Ensure your changes align with the project's contribution guidelines.
5. **Commit Your Changes**:
    - Stage your changes and commit them with a descriptive message:
      ```bash
      git add .
      git commit -m "Your descriptive message"
      ```
6. **Push Your Changes:**
    - Push your branch to your forked repository:
      ```bash
      git push origin your-branch-name
      ```
7. **Create a Pull Request (PR):**
    - Go to the original repository on GitHub and click “Compare & pull request.” Provide a clear description of the changes and submit the PR.

Once your PR is reviewed and approved, it will be merged into the main branch.
    

---

# Contributors

| Avatar | Contributor | GitHub Profile | No of Contributions |
|:--------:|:--------------:|:----------------:|:-------------------:|
| <img src='https://avatars.githubusercontent.com/u/100434825?v=4' width='40' height='40' style='border-radius:50%;'/> | Eemayas | [@Eemayas](https://github.com/Eemayas) | 14 |

    

---

# License

This project is licensed under the MIT License - see the [LICENSE](./LICENSE) file for details.


---


<!-- @format -->

### Save README to File


In [44]:
# Specify the file name
file_name = f"output/{metadata.name}_README.md"

# Open the file in write mode with utf-8 encoding and save the content
with open(file_name, "w", encoding="utf-8") as file:
    file.write(str(combined_markdown))

print(f"{file_name} has been created and saved.")

output/Daraz_Scraper_README.md has been created and saved.
